In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters

import pdfplumber


In [4]:
os.getcwd()
#os.chdir("/Users/veesheen/Desktop")

'/Users/veesheenyuen/code/veeyuen/SAA'

# Development Code

In [271]:
if 'Disqualified' in fragment:
    fragment.insert(7, ' ')
    
else:
    fragment.insert(len(fragment), ' ')

In [573]:
import re

pattern='[A-Z]{2,5}'

for i in range(len(fragment)):

    if re.search(pattern, fragment[i]):
        print(i, re.search(pattern, fragment[i]))
        index=i  # set index to second value if there are two uppercase values encountered
        print(index)



1 <re.Match object; span=(0, 4), match='CHIJ'>
1
6 <re.Match object; span=(0, 3), match='SNG'>
6


In [384]:
# Regex pattern testing

#pattern='([A-Z][a-z]+\s){2,4}'

#pattern='\d{1,3}{(?<=\s)[A-Z]{3,5}'

#pattern='[A-Z]{2,4}'
#pattern='\d\s.{5,40}?\s(Q\s|q\s|\d)'


pattern='\d\s\[A-Z]{3,}(?=Q|q|[0-9]{2,3})'


#string='8 CHIJ Sec (Toa Payoh) HIJ 2 7 00:54.31'
#string='999 Chestnut Drive Sec CDS 2 3 Withdrawal'
#string='7 CHIJ Katong Convent KC 5 7 00:54.10'
#string = '1 XIN BOWEN 587 HCI Finals 6 40.15 9'



#string='2 GUOK XUE QIAN 143 SNG Finals 6 8.91 7'


#matches=re.finditer(pattern, string)

matches=re.findall(pattern, string)

print(matches)

#print(type(matches))

#for match in matches:
#    print(match)



[]


In [126]:
# Test splicing and extracting name

#s = "4 PHOEBE ANG KE WEI Q 800 CGS h4 2 01:05.62"
#s="2 Sarah Chong Xin RI 1 12 4.78 7"
#s='7 CHIJ Katong Convent KC 5 7 00:54.10'
#s='4 WANG HANCHEN 584 HCI Finals 1 36.79 5'
#s='1 XIN BOWEN 587 HCI Finals 6 40.15 9'
#s='3 SINGAPORE SPORTS SCHOOL Q 598 SSP h1 3 04:32.84'
s='16 TEMASEK SECONDARY SCHOOL 873 TMS h4 3 05:08.64'

lpos = re.search('\s', s)
#rpos = re.search('\b[A-Z]{2-4}\b\d', s)

#rpos = re.search('\sQ\s', s)
#rpos = re.search('\s[A-Z]{2,4}?', s)
#rpos = re.search('\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?|\s[A-Z]{2,4}\s\d?', s)
rpos = re.search('\d\s.{5,40}?\s(Q|q|\d)', s)



split_pos_end=rpos.end()-1
split_pos_start=lpos.start()+1



l = s[:lpos.start()]
r = s[rpos.start():]

#name=s[lpos.start():rpos.start()]

name=s[split_pos_start:split_pos_end]

print(name)

TEMASEK SECONDARY SCHOOL 


In [87]:
# test regex for event name extraction

s='Event: S13-09 Discus - B DIVISION BOYS Finals'
event_pattern=':\s.{5,20}\s\-'


re.search(event_pattern, s)


<re.Match object; span=(5, 22), match=': S13-09 Discus -'>

In [38]:
# Test pdfplumber

#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/sectrack_audit_2010_1-1')
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/')
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/') 
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/') 


import pdfplumber

#file="sectrack_audit_2010_211-211.pdf"
#file="Session 13_13-end.pdf"
#file="Session 13_1-4.pdf"
#file="sectrack_audit_2010_28-28.pdf"
#file="Session 13_5-6.pdf"
#file='Session 13_5-6_2-end.pdf'   # stranded pdf
file='SGO24.pdf'


#table_settings = {
#    "vertical_strategy": "text",
#    "horizontal_strategy": "text",
#    }


with pdfplumber.open(file) as pdf:
    
    page = pdf.pages[0]
    table=page.extract_table()
    text=page.extract_text()


splitted=text.splitlines()

splitted

["Singapore Athletics Assoc.-Open Champ - Organization License Hy-Tek's MEET MANAGER 9:03 PM 19 Apr 2024 Page 1",
 '84th Singapore Open Track & Field - 18 Apr 2024 to 19 Apr 2024',
 'Championships 2024',
 'Singapore Sportshub',
 'Results',
 'Event 101 Women Shot Put Open',
 '=========================================================================',
 'Name Age Team Seed Finals',
 '=========================================================================',
 'Flight 1',
 '1 # 154 Intadis, Areerat 28 Thailand 16.71m 14.66m',
 '13.78m 14.41m X 14.29m 14.34m 14.66m',
 '2 # 84 Yee Wai Teng, Mel 26 Singapore 12.01m 10.60m',
 'X 10.33m X 10.16m 10.60m 10.44m',
 '3 # 172 Neo, Ee Nin 18 Wings Athlet 9.21m 9.32m',
 '8.31m 9.30m 9.17m 8.56m 9.32m 8.59m',
 'Event 102 Women 1500 Meter Run Open',
 '=========================================================================',
 'Name Age Team Seed Finals',
 '=========================================================================',
 'Section 1',
 '1 # 4

In [210]:
pd.DataFrame(splitted)

,0
0,23 BUKIT VIEW SECONDARY 999 BTV h3 4 05:51.77
1,SCHOOL
2,/


In [35]:
def old_assign(table):

    global index
    
    exclusions=[]
    
    df=pd.DataFrame(table)
    columns=df.iloc[6,0]
    names=columns.split(' ')

    df2 = pd.DataFrame(columns=names)
    
    new_df = df2.iloc[:, :-2]    # remove last 2 extraneous columns
     
    
    for i in range(7,(len(df)-1)):
            
            
        if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)

            
            continue

            
        s=table[i]
        
        lpos = re.search('\s', s)
        #rpos = re.search('\sQ\s', s)
        #rpos = re.search('\s[A-Z]{2,4}', s)
        
        matches=re.findall('\s[A-Z]{2,4}', s)  # find how many uppercase shortforms for school
        
        if len(matches)==2:
            rpos=re.search(matches[1], s)
        else:
            rpos = re.search('\s[A-Z]{2,4}', s)

            
        # Splice string to extract name of competitor

            
        l = s[:lpos.start()] # left string post splicing
        r = s[rpos.start():] # right string post slicing
         
        name=s[lpos.start():rpos.start()] # extract competitor name
                
        list=r.split(' ')        

        combined=l+ ',' + name + ', '.join(list)

        row=combined.split(",")
                        
        if len(row)!=6:
            
            n=len(row)-6
            row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns                 
                    
        new_df.loc[len(new_df)] = row

    return new_df

In [36]:
old_assign(splitted)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/484862590.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/484862590.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/484862590.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,Pos,Competitor,Team,Ht,Ln,Result
0,1,Shawn Poh Zhi Kang,SKS,13,7,00:24.20
1,2,Liaw Jian Hui,AMK,3,7,00:24.68
2,3,Zubin Percy Muncherji,ACS(BR),8,7,00:24.77
3,4,Terence Loh Junheng,ACS(I),2,4,00:24.84
4,5,Md Nur Syahirran,SSP,10,3,00:24.90
5,6,Ong Sin Yao,CCH,2,7,00:24.97
6,7,Wu Chee Keen Daniel,DSS,9,1,00:25.05
7,8,Jonathan Lew Lian Xin,RI,11,5,00:25.06
8,9,Ang Li Sheng,AND,2,6,00:25.16
9,10,Teo Yu Ming,RI,6,6,00:25.62


In [73]:
# Old version- does not capture REMARKS and NR columns

def new_format_parser(row_index, names, table, master_df):
            
    # Define regex patterns
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    stranded = "no"  # flag for stranded pdf pages

        
    # Put table that has been processed into a dataframe
    
    df=pd.DataFrame(table)
            
    columns=[]
    
    
    # Initialize new df and remove unnecessary columns
    
    if row_index!=0 and names is not None: #check that its coming from not a stranded pdf with no header
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns
    
    else:
        
        print('reached this first point for a stranded pdf')
        
        stranded="yes"
        
        new_df = master_df.iloc[:0] # new_df columns to come from the master df
        
        row_index=-1  # move back row_index for a stranded pdf
        
    
#    for i in range(row_index+1,(len(df)-1)):  # iterate over df - only stranded pdf should start from zero. Previous code - revert to original  


#  check for stranded pdfs with stranded names

    for i in range(row_index+1,(len(df))):  # iterate over df - only stranded pdf should start from zero        

        
        if 'DNM' in table[i]:
                    
            continue # skip row if DNM, DQ, DNF etc.
                                    
        if not re.search('^\d', table[i]): # look for stranded parts of names (does not begin with a digit). But need to test for standed names in stranded pdf
        
        
            try:  # remove try block to revert back to previous working state
                
                row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name. Stranded pdf will throw and error here
                
                new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                        
                new_df.loc[len(new_df)] = row  # add back amended row
                
                print(row)
                                    
                continue  # then skip to next row
            
            except:
                
                print('reached this second point for a stranded pdf')
                
                pass                 # move on if it is actually a stranded pdf
                              
                
        string=table[i] 
        
        print(string)
                                
        lpos = re.search('\s', string)
        rpos = re.search(new_pattern, string)
                                            
            
        # Splice string to extract name of competitor

        split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
        split_pos_start=lpos.start()+1

            
        left_string = string[:lpos.start()] # left string post splicing
        right_string = string[split_pos_end:] # right string post slicing
                         
        name=string[split_pos_start:split_pos_end]  # extract competitor name
                                        
        list=right_string.split(' ')        # put everything coming after the name into a list   
                
                
        if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
            list.extend([' ', ' '])
                    
        elif len(list)==5:
                    
            list.extend([' ', ' '])
                                                            
                
        combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

        row=combined.split(';')        
                                                                             

        # check if Q or q between tag and name. If not, fill in with empty space.
        # don't overwrite master df if there is a stranded pdf
                
        if row[2].strip()=='Q' or row[2].strip()=='q' or row[2] is None :
                    
            new_df.loc[len(new_df)] = row
                        
#            master_df=new_df      # previous

        else:
                    
            row.insert(2, ' ')  # insert a blank field if no Q or q    
            
            print(row)
                
            new_df.loc[len(new_df)] = row
            
            
# QAed to this point for a stranded pdf. Need to figure out how to attach stranded row to master df
            
            
#            master_df=new_df      # previous   

    
    return new_df

In [ ]:
def extraction(table, master_df):  # second latest version

    global index, row_index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    session_pattern='\d\d-\d\d'
    
    event = None
    
    names = None
    
    temp_df = pd.DataFrame()

    
    # Put extracted info from pdf into a df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
        
    for index, row in df.iterrows(): # find row containing event details and column names
                
        
        if 'Event' in row[0]:
                                                
            epos = re.search(event_pattern, row[0])
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=event[split_start:split_end]
                                    
            
            if 'BOYS' in event or 'Boys' in event:
                gender='Male'
                
            if 'GIRLS' in event or 'Girls' in event:
                gender='Female'    
                
            
            if 'Finals' in event:
                round='Finals'
            
            region='Local'
            
            
            session=re.findall(session_patttern, row[0])
            
            print('session', session)
                  
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
            
    
    if (event and names) is None:  # if it is a stranded page (no header) that is cutoff from the main pdf
                                # Change this code to add iteration over pages. Just append splitted 2nd page.
                                
            temp_df = master_df.iloc[:0]  # initiate empty df and copy column names
                                    
            row_index=0   # if no header 
            
            names = None  # retrieve master df and append instead
            
            print('here')
            
            print(table)
                                            
            temp_df=new_format_parser(row_index, names, table, master_df) # need to concat master_df and temp df
            
            print('reached this 3rd point for a stranded pdf')
            
            master_df= pd.concat([master_df, temp_df])  # this concat is not working as temp_df is empty
                    
            return master_df # exit function

            
    
    # Check pdf format and begin extraction of results
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
            
        return new_df
            
    elif 'W/G' in columns: # new format 
    
            
        master_df=new_format_parser(row_index, names, table, master_df) 
     
    ##        master_df= pd.concat([temp_df, master_df])
        
#        print(master_df)
    
    return master_df

In [97]:
def extraction_old(table, master_df):  # old version without calling new_format_parser function

    global index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'

    
    # Initialize new df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
    
    for index, row in df.iterrows(): # find row containing event details and column names
        
        
        if 'Event' in row[0]: # data is in column '0' 
            
            event=row[0]   # set string for extraction of event info
            
                        
            epos = re.search(event_pattern, event)
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=event[split_start:split_end]
                                    
            
            if 'BOYS' in event or 'Boys' in event:
                gender='Male'
                
            if 'GIRLS' in event or 'Girls' in event:
                gender='Female'    
                
            
            if 'Finals' in event:
                round='Finals'
            
            region='Local'  
            
       
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
            
    
    
    
    # Check pdf format and begin extraction of results
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
        return new_df
            
    elif 'W/G' in columns: # new format 
            
            df2 = pd.DataFrame(columns=names)        
    
            new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns        

         
            for i in range(row_index+1,(len(df)-1)):
                                
                if 'DNM' in table[i]:
                    
                    continue # skip row if DNM, DQ, DNF etc.
                            
                if not re.search('^\d', table[i]): # look for stranded parts of names (does not begin with a digit)
            
                    row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name
                
                    new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                        
                    new_df.loc[len(new_df)] = row  # add back amended row
                    
                    continue  # then skip to next row
                              

                string=table[i]
                                
                lpos = re.search('\s', string)
                rpos = re.search(new_pattern, string)
                                            
            
        # Splice string to extract name of competitor

                split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name            
                                
                list=right_string.split(' ')        # put everything coming after the name into a list   
                
                
                if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
                    list.extend([' ', ' '])
                    
                elif len(list)==5:
                    
                    list.extend([' ', ' '])
                                                            
                
                combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

                row=combined.split(';')
                                                                                
                
        # check if Q or q between tag and name. If not, fill in with empty space.
        
                
                if row[2].strip()=='Q' or row[2] is None:
                    
                    pass
                    
                else:
                    
                    row.insert(2, ' ')  # insert a blank field if no Q or q
                    
                                            
                new_df.loc[len(new_df)] = row
                
                print(row)
                    
                    
            return new_df

In [40]:
master_df=None

master_df=extraction_old(splitted, master_df)  # using old extraction function

0 Singapore Schools Sports Council 14-04-2010
1 51st National Inter-School Track and Field Championships 2010 10:02 PM
2 Audit List Page 28
3 Event : 200 Metres (C-Boys) Type : All Heats
4 02-03
5 Record : Mohd Yusof Bin Azhari SJI 2003 ET 00:22.99s
6 Pos Competitor Team Ht Ln Result Pts Remark
7 1 Shawn Poh Zhi Kang SKS 13 7 00:24.20
8 2 Liaw Jian Hui AMK 3 7 00:24.68
9 3 Zubin Percy Muncherji ACS(BR) 8 7 00:24.77
10 4 Terence Loh Junheng ACS(I) 2 4 00:24.84
11 5 Md Nur Syahirran SSP 10 3 00:24.90
12 6 Ong Sin Yao CCH 2 7 00:24.97
13 7 Wu Chee Keen Daniel DSS 9 1 00:25.05
14 8 Jonathan Lew Lian Xin RI 11 5 00:25.06
15 9 Ang Li Sheng AND 2 6 00:25.16
16 10 Teo Yu Ming RI 6 6 00:25.62
17 11 Clyde Tan Jin Shuen ACS(I) 8 6 00:25.68
18 12 Chua Cheng Ying SJI 13 6 00:25.69
19 13 Puah Li An NTS 4 5 00:25.70
20 14 Raymond Scott Lee Chian Hoong SJI 2 5 00:25.84
21 15 Ridwan Bin Yahya TMS 1 8 00:25.85
22 16 Tan Hui Ming Zac NSS 5 8 00:25.89
23 17 Daryl Shane Lee TMS 5 5 00:25.93
24 18 Chin Yew 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/2665074277.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/2665074277.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/2665074277.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [30]:
master_df

,Pos,Competitor,Team,Ht,Ln,Result
0,1,Singapore Sports School,SSP,1,4,00:49.67
1,2,CHIJ St Nicholas Girls' School,SNG,1,5,00:51.41
2,3,Nanyang Girls' High School,NYGH,1,6,00:52.25
3,4,Cedar Girls' Sec School,CG,1,3,00:52.90
4,5,CHIJ Sec (Toa Payoh),HIJ,1,7,00:53.88
5,6,CHIJ Katong Convent,KC,1,2,00:55.58
6,7,North Vista Sec School,NV,1,8,00:55.69
7,8,Raffles Girls' School (Sec),RGS,1,1,00:57.27


In [ ]:
# Test iteration over more than one page

#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/Deconstructed/S01-01_TO_11_deconstructed.pdf"

 
# Iterate over files in directory

#sorted_items=sort_directory(directory)

master_df=pd.DataFrame()  # initialize empty master df

splitted=None

with pdfplumber.open(file) as pdf:
    
    for i in range(len(pdf.pages)):
        
        temp=splitted
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()
        
        if i==0:
        
            splitted=text.splitlines()            
            
        else: # stitch list from second page
            
            temp=text.splitlines()
            
            splitted.extend(temp)
            
    master_df=extraction(splitted, master_df)

                


In [ ]:
# To do:
# 1. code old_format_parser function for old format pdfs
# 2. Test iteration over several pages using pdf.pages[0]
# 3. Modify code to iterate over second stranded page and add data

# Extract list of schools

In [341]:
# Extract list of primary and secondary schools
# Using BS4

URL ='https://en.wikipedia.org/wiki/List_of_secondary_schools_in_Singapore'
page=requests.get(URL)

soup=BeautifulSoup(page.content, 'html.parser')


In [342]:
# Creating list with all tables
tables = soup.find_all('table')

#  Looking for the table with the classes 'wikitable' and 'sortable'
table = soup.find('table', class_='wikitable')

In [343]:
for a in soup.find_all('table'):
    print(a.text)




Name
Type
SchoolCode
Area[4]
Notes / Affiliations (Admissions / Academic Programmes)
Website


Admiralty Secondary School
Government
3072
Woodlands

[1]


Ahmad Ibrahim Secondary School
Government
3201
Yishun

[2]


Anderson Secondary School
GovernmentAutonomous
3001
Ang Mo Kio

[3]


Anglican High School
Government-aidedAutonomousSAP

7101
Bedok

[4]


Anglo-Chinese School (Barker Road)
Government-aided
7032
Newton
Affiliated to:
Methodist Girls' School (Secondary)

[5]


Anglo-Chinese School (Independent)
IndependentIP
IP:9161Express:7001

Queenstown

Offers the International Baccalaureate certificate
Affiliated to:
Methodist Girls' School (Secondary) (IP)

[6]


Ang Mo Kio Secondary School
Government
3026
Ang Mo Kio

[7]


Assumption English School
Government-aided
7002
Bukit Panjang

Affiliated to:
Catholic Junior College (Non-IP)[5]

[8]


Bartley Secondary School
Government
3002
Toa Payoh

[9]


Beatty Secondary School
Government
3003
Toa Payoh

[10]


Bedok Green Secondary Sch

In [364]:
# Extract list of schools using Pandas read_html

sschool_html = pd.read_html("List of secondary schools in Singapore - Wikipedia.html")
pschool_html = pd.read_html("List of primary schools in Singapore - Wikipedia.html")

In [357]:
sschool_html[0]['Name']

0              Admiralty Secondary School
1          Ahmad Ibrahim Secondary School
2               Anderson Secondary School
3                    Anglican High School
4      Anglo-Chinese School (Barker Road)
                      ...                
140                Yuhua Secondary School
141          Yusof Ishak Secondary School
142               Yuying Secondary School
143             Zhenghua Secondary School
144             Zhonghua Secondary School
Name: Name, Length: 145, dtype: object

In [376]:
sec_schools=sschool_html[0]['Name'].tolist()

In [377]:
sec_schools

['Admiralty Secondary School',
 'Ahmad Ibrahim Secondary School',
 'Anderson Secondary School',
 'Anglican High School',
 'Anglo-Chinese School (Barker Road)',
 'Anglo-Chinese School (Independent)',
 'Ang Mo Kio Secondary School',
 'Assumption English School',
 'Bartley Secondary School',
 'Beatty Secondary School',
 'Bedok Green Secondary School',
 'Bedok South Secondary School',
 'Bedok View Secondary School',
 'Bendemeer Secondary School',
 'Boon Lay Secondary School',
 'Bowen Secondary School',
 'Broadrick Secondary School',
 'Bukit Batok Secondary School',
 'Bukit Merah Secondary School',
 'Bukit Panjang Government High School',
 'Bukit View Secondary School',
 'Catholic High School',
 'Canberra Secondary School',
 "Cedar Girls' Secondary School",
 'Changkat Changi Secondary School',
 'CHIJ Katong Convent (Secondary)',
 'CHIJ Secondary (Toa Payoh)',
 "CHIJ St. Joseph's Convent",
 "CHIJ St. Nicholas Girls' School",
 "CHIJ St. Theresa's Convent",
 'Chua Chu Kang Secondary School',
 

In [367]:
pschool_html[1]

,Name,Funding,Type,Area[3],Notes,Website,School Code
0,Admiralty Primary School,Government,Mixed,Woodlands,NaN,[1],1744
1,Ahmad Ibrahim Primary School,Government,Mixed,Yishun,NaN,[2],1738
2,Ai Tong School,"Government-aided, SAP",Mixed,Bishan,Affiliated to Singapore Hokkien Huay Kuan[4],[3],5625
3,Alexandra Primary School,Government,Mixed,Bukit Merah,NaN,[4],1266
4,Anchor Green Primary School,Government,Mixed,Sengkang,NaN,[5],1254
...,...,...,...,...,...,...,...
180,Yuhua Primary School,Government,Mixed,Jurong East,NaN,[180],1656
181,Yumin Primary School,Government,Mixed,Tampines,NaN,[181],1219
182,Zhangde Primary School,Government,Mixed,Bukit Merah,NaN,[182],1199
183,Zhenghua Primary School,Government,Mixed,Bukit Panjang,NaN,[183],1240


In [374]:
prim_schools=pschool_html[1]['Name'].tolist()

In [375]:
prim_schools

['Admiralty Primary School',
 'Ahmad Ibrahim Primary School',
 'Ai Tong School',
 'Alexandra Primary School',
 'Anchor Green Primary School',
 'Anderson Primary School',
 'Anglo-Chinese School (Junior)',
 'Anglo-Chinese School (Primary)',
 'Angsana Primary School',
 'Ang Mo Kio Primary School',
 'Beacon Primary School',
 'Bedok Green Primary School',
 'Bendemeer Primary School',
 'Blangah Rise Primary School',
 'Boon Lay Garden Primary School',
 'Bukit Panjang Primary School',
 'Bukit Timah Primary School',
 'Bukit View Primary School',
 'Canberra Primary School',
 'Canossa Catholic Primary School',
 'Cantonment Primary School',
 'Casuarina Primary School',
 'Catholic High School (Primary)',
 'Cedar Primary School',
 'Changkat Primary School',
 'CHIJ (Katong) Primary',
 'CHIJ (Kellock)',
 'CHIJ Our Lady of Good Counsel',
 'CHIJ Our Lady of the Nativity',
 'CHIJ Our Lady Queen of Peace',
 'CHIJ Primary (Toa Payoh)',
 "CHIJ St. Nicholas Girls' School (Primary Section)",
 'Chongfu School'

In [378]:
all_schools = prim_schools+sec_schools

In [379]:
all_schools

['Admiralty Primary School',
 'Ahmad Ibrahim Primary School',
 'Ai Tong School',
 'Alexandra Primary School',
 'Anchor Green Primary School',
 'Anderson Primary School',
 'Anglo-Chinese School (Junior)',
 'Anglo-Chinese School (Primary)',
 'Angsana Primary School',
 'Ang Mo Kio Primary School',
 'Beacon Primary School',
 'Bedok Green Primary School',
 'Bendemeer Primary School',
 'Blangah Rise Primary School',
 'Boon Lay Garden Primary School',
 'Bukit Panjang Primary School',
 'Bukit Timah Primary School',
 'Bukit View Primary School',
 'Canberra Primary School',
 'Canossa Catholic Primary School',
 'Cantonment Primary School',
 'Casuarina Primary School',
 'Catholic High School (Primary)',
 'Cedar Primary School',
 'Changkat Primary School',
 'CHIJ (Katong) Primary',
 'CHIJ (Kellock)',
 'CHIJ Our Lady of Good Counsel',
 'CHIJ Our Lady of the Nativity',
 'CHIJ Our Lady Queen of Peace',
 'CHIJ Primary (Toa Payoh)',
 "CHIJ St. Nicholas Girls' School (Primary Section)",
 'Chongfu School'

# Test scannning of directory files and opening

In [192]:
# Test scan of directory files and opening

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/"
 
# Iterate over files in directory
for filename in os.scandir(directory):
    # Open file
    with open(os.path.join(directory, filename)) as f:
        print(f"Content of '{filename}'")    
    
    with pdfplumber.open(file) as pdf:
    
        page = pdf.pages[0]
        table=page.extract_table()
        text=page.extract_text()


    splitted=text.splitlines()

    splitted

Content of '<DirEntry 'Session 13_5-6_2-end.pdf'>'
Content of '<DirEntry 'Session 13_5-6_1-1.pdf'>'


In [243]:
splitted

['Audited List',
 'Event: S13-03 4 X 400m Relay - B DIVISION GIRLS Heats',
 'Schools National Record Singapore Sports School SSP (ET)04:01.98s 2011',
 'Championships Record Singapore Sports School SSP (ET)04:01.98s 2011',
 'POS Competitor Q Tag Team Heat Ln Result Pts W/G Remarks NR',
 '1 SINGAPORE SPORTS SCHOOL Q 720 SSP h3 3 04:23.41',
 "2 NANYANG GIRLS' HIGH Q 1022 NYGH h3 6 04:24.02",
 'SCHOOL',
 "3 CRESCENT GIRLS' SCHOOL Q 791 CGS h3 2 04:24.09",
 "4 CEDAR GIRLS' SECONDARY Q 871 CG h2 4 04:25.16",
 'SCHOOL',
 "5 CHIJ ST. NICHOLAS GIRLS' Q 656 SNG h2 5 04:30.99",
 'SCHOOL (SECONDARY)',
 '6 NATIONAL JUNIOR COLLEGE Q 810 NJC h2 8 04:32.69',
 '7 DUNMAN HIGH SCHOOL Q 930 DHS h2 2 04:37.82',
 '8 CHIJ SECONDARY (TOA PAYOH) Q 747 HIJ h3 5 04:39.52',
 "9 RAFFLES GIRLS' SCHOOL Q 845 RGS h2 1 04:40.60",
 '(SECONDARY)',
 '10 AHMAD IBRAHIM SECONDARY 648 AIS h1 6 04:43.48',
 'SCHOOL',
 '11 PAYA LEBAR METHODIST 953 PLMGS h3 1 04:48.28',
 "GIRLS' SCHOOL (SECONDARY)",
 '12 NORTH VISTA SECONDARY 70

In [477]:
def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items

In [246]:
sort_directory(directory)

['Session 13_5-6_1-1.pdf', 'Session 13_5-6_2-end.pdf']

In [ ]:
# Test open all files in directory

def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items


directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022PDF/"
 
# Iterate over files in directory

sorted_items=sort_directory(directory)

print(sorted_items)

sorted_items.pop(0)

master_df=pd.DataFrame()  # initialize empty master df

for file in sorted_items:
    
    print(file)
    
    temp = pd.read_csv(file)
    
    master_df=pd.concat([master_df, temp], axis=0)
    


In [78]:
# This is the starting point. Need 'extraction' and 'new_format_parser' functons.
# Test scan of directory with multiple files and opening 


#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/')

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/Deconstructed/"
 
# Iterate over files in directory

sorted_items=sort_directory(directory)

master_df=pd.DataFrame()  # initialize empty master df

for file in sorted_items:
    
    print(file)

    with pdfplumber.open(file) as pdf:
    
        page = pdf.pages[0]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()


    splitted=text.splitlines() # if more than one page should append the two 'splitted' files before extraction
            
    master_df=extraction(splitted, master_df)
    
    


S01-01_TO_11_deconstructed.pdf


FileNotFoundError: [Errno 2] No such file or directory: 'S01-01_TO_11_deconstructed.pdf'

In [126]:
splitted

['23 BUKIT VIEW SECONDARY 999 BTV h3 4 05:51.77', 'SCHOOL', '/']

In [271]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G
0,1,SINGAPORE SPORTS SCHOOL,Q,720,SSP,h3,3,04:23.41,,
1,2,NANYANG GIRLS' HIGH SCHOOL,Q,1022,NYGH,h3,6,04:24.02,,
2,3,CRESCENT GIRLS' SCHOOL,Q,791,CGS,h3,2,04:24.09,,
3,4,CEDAR GIRLS' SECONDARY SCHOOL,Q,871,CG,h2,4,04:25.16,,
4,5,CHIJ ST. NICHOLAS GIRLS' SCHOOL (SECONDARY),Q,656,SNG,h2,5,04:30.99,,
5,6,NATIONAL JUNIOR COLLEGE,Q,810,NJC,h2,8,04:32.69,,
6,7,DUNMAN HIGH SCHOOL,Q,930,DHS,h2,2,04:37.82,,
7,8,CHIJ SECONDARY (TOA PAYOH),Q,747,HIJ,h3,5,04:39.52,,
8,9,RAFFLES GIRLS' SCHOOL (SECONDARY),Q,845,RGS,h2,1,04:40.60,,
9,10,AHMAD IBRAHIM SECONDARY SCHOOL,,648,AIS,h1,6,04:43.48,,


In [154]:
# To do:
# 1. code old_format_parser function for old format pdfs
# 2. Test iteration over several pages using pdf.pages[0]
# 3. Modify code to iterate over second stranded page and add data

""


# Extract 2024 NSG PDFs

In [5]:
# ONLY For 2024 NSG pdfs--latest version
#
# New schema
#df_filtered = df_filtered.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
#                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
#                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'DATE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'RX_TIME'])


os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024PDF/sectrack_result_02p.pdf"

 
# Iterate over files in directory

#sorted_items=sort_directory(directory)

new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
#event_pattern='\:\s.{5,30}\s(\-|\()'

event_pattern='(\:\s.{5,32})\sT'

final_pattern = '.{4,30}\('


school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

session_pattern='\d\d\-\d\d'

div_pattern='([A-Z]\-Boys)|([A-Z]\-Girls)'

wind_pattern='(H[0-9])\s\:\s(\+\d\.\d|\-\d\.\d)'

event_mapping={'100m': 'Sprint', '100m': 'Sprint'}    


df_table=pd.DataFrame()  # initialize empty master df

df=None



with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for index, row in df.iterrows(): # find row containing event details and column names
                
        
            if 'Event' in row[0]:
            
                details=row[0]
                                                
                if 'BOYS' in details or 'Boys' in details:
                    gender='Male'
                
                if 'GIRLS' in details or 'Girls' in details:
                    gender='Female'    
                
                # Slice event name. Must be done in 2 stages.   
                
                
                epos = re.search(event_pattern, details)  
                                
                        
                split_end=epos.end()-1      # adjust the splicing position to capture the event + division
                split_start=epos.start()+1
    
                event=details[split_start:split_end]
                
                pos=re.search(final_pattern, event)  # new
            
                final_end=pos.end()-1      # new
                final_start=pos.start()+1  # new
                
                final_match=event[final_start:final_end] # finally extract event name
                
                session_list=re.findall(session_pattern, details)
                
                session=session_list[0]
                
                print('session', session)
                
                
                
            # Assign high level event type
                
                if 'Throw' in final_match:
                    category_event='Throw'
                elif 'Jump' in final_match:
                    category_event='Jump'
                elif 'Vault' in final_match:
                    category_event='Jump'
                elif '100' in final_match:
                    category_event='Sprint'
                elif '1500' in final_match:
                    category_event='Mid'
                elif 'Javelin' in final_match:
                    category_event='Throw'
                elif 'Shot' in final_match:
                    category_event='Throw'
                elif '400' in final_match:
                    category_event='Sprint'
                elif '800' in final_match:
                    category_event='Mid'
                elif '400' in final_match:
                    category_event='Sprint'
                elif 'Hurdles' in final_match:
                    category_event='Hurdles'
                elif '200' in final_match:
                    category_event='Sprint'
                elif 'Walk' in final_match:
                    category_event='Walk'
                elif '3000' in final_match:
                    category_event='Long'
                elif '5000' in final_match:
                    category_event='Long'
                elif '2000' in final_match:
                    category_event='Mid'
                elif 'Discus' in final_match:
                    category_event='Throw'
                
                if 'Relay' in final_match: # Change from Sprint to Relay for relays only
                    category_event='Relay'

               
                # Slice division
        
                divpos = re.search(div_pattern, details)
                            
                s_pos=divpos.start()
                e_pos=divpos.start()+1
    
                div=details[s_pos:e_pos] # extract division        
                                      
            
            if 'Competitor' in row[0]: # extract column names
            
                row_index=index  # where the header row is
                columns=df.iloc[row_index,0]
                names=columns.split(' ')
                
            if 'Final' in row[0]:
                
                stage="Final"
                
            if 'Heat' in row[0]:
                
                stage='Heats'
                
            if 'SF' in row[0]:
                
                stage = 'SF'    
                
                               
            if 'W/Gauge' in row[0]:
                
                details=row[0]            
                
                try:
            
                    wind=re.findall(wind_pattern, details)
                                    
                    wind_dict = {}
                    
                    for key, val in wind: # create dictionary of extracted wind data
                        wind_dict.setdefault(key, val)
                
                except:
                    
                    pass
            
                    
        temp=pd.DataFrame(table)
        
                            
                
        temp.iloc[2:,:]  # drop first 2 rows

        temp['EVENT'] = final_match
        temp['GENDER'] = gender
        temp['DIVISION'] = div
        temp['STAGE'] = stage
        temp['COMPETITION'] = 'NSG'
        temp['REGION'] = 'Local'
        temp['CATEGORY_EVENT'] = category_event
        temp['test'] = 'H'+temp[5].map(str) # create a new temp field to map across wind
        temp['SOURCE'] = 'https://nsg.moe.edu.sg/results'       
        temp['YEAR'] = '2024'
        temp['QUALIFICATION']=''
        temp['POINTS']=''
        temp['UNIQUE_ID']=''
        temp['COUNTRY']=''
        temp['DICT_RESULTS']=''
        temp['DOB']=''
        temp['GROUP']=''
        temp['ATHLETE_ID']=''
        temp['TIMESTAMP']=''
        temp['VENUE']=''
        temp['DATE']=session
        temp['SUB_EVENT']=''   # event=decathlon, sub_event=shot put
        temp['SESSION']=''
        temp['EVENT_CLASS']=''   # weight height category for javelin, high jump etc.
        temp['RX_TIME']=''
        
        
        
        
        
# need to add 'VENUE', 'DATE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'RX_TIME' for new updated schema        
        
        
        temp['WIND']=temp['test'].map(wind_dict) # map temp field across to wind data     
        
        
        temp = temp[temp[0].str.contains("~~~ End of Listing ~~~") == False]  # drop useless rows
        temp = temp[temp[0].str.contains("W/Gauge :") == False]
        temp = temp[temp[0].str.contains("Pos") == False]
     #   temp = temp[temp[7].str.contains("DNS|NM|DQ|DNF") == False] # drop rows without results
        
            
        df_table=pd.concat([df_table, temp], axis=0)
         
        


df_table.rename(
    columns={0: "RANK", 1: "TAG_ID", 2: "NAME", 3: "TEAM", 4: "RESULT", 5: "HEAT", 6: "LANE", 7: "REMARKS"},
    inplace=True,
)
        
df_table.drop('test', axis=1, inplace=True)    # drop temp columns used to map wind data   

df_table = df_table.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'DATE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'RX_TIME'])

# Map NM/DNS/DQ/DNF to results column                            
                            
mask = df_table['REMARKS'].str.contains(r'NM', na=True)
df_table.loc[mask, 'RESULT'] = 'NM'

mask = df_table['REMARKS'].str.contains(r'DNS', na=True)
df_table.loc[mask, 'RESULT'] = 'DNS'
                            
mask = df_table['REMARKS'].str.contains(r'DQ', na=True)
df_table.loc[mask, 'RESULT'] = 'DQ'

mask = df_table['REMARKS'].str.contains(r'DNF', na=True)
df_table.loc[mask, 'RESULT'] = 'DNF'
                            
                            

df_table.to_csv('sectrack_result_01p.csv', sep=',', index=False, encoding='utf-8')
        

            
                       
                       

        
            
                


session 02-01
session 02-02
session 02-03
session 02-04
session 02-05
session 02-06
session 02-07
session 02-07
session 02-07
session 02-08
session 02-08
session 02-09
session 02-09
session 02-10
session 02-10
session 02-11
session 02-11
session 02-11
session 02-12
session 02-12


In [11]:
df_table

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,REACT_TIME,SESSION,FREE_FIELD3
2,1,662,Tan Shou Yi Rei (Chen Shouyi),RI,NaN,13.66,,2,9,NaN,...,,,Jump,,https://nsg.moe.edu.sg/results,Raffles Institution,,,02-01,
3,2,879,"Tang Kai Sheng, Cayman",VS,NaN,13.15,,1,4,NaN,...,,,Jump,,https://nsg.moe.edu.sg/results,Victoria School,,,02-01,
4,3,642,Lau Jia Hern,RI,NaN,12.86,,1,11,NaN,...,,,Jump,,https://nsg.moe.edu.sg/results,Raffles Institution,,,02-01,
5,4,881,Telukula Sourendra,VS,NaN,12.59,,2,4,NaN,...,,,Jump,,https://nsg.moe.edu.sg/results,Victoria School,,,02-01,
6,5,757,"Low Ming Dao, Jase",SSP,NaN,12.32,,1,8,NaN,...,,,Jump,,https://nsg.moe.edu.sg/results,Singapore Sports School,,,02-01,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,38,398,Reuben Chiang,TMJC,NaN,05:19.65,,2,7,NaN,...,,,Mid,,https://nsg.moe.edu.sg/results,Tampines Meridian Junior,,,02-12,
40,39,232,Peh Kang Shen Javis,EJC,NaN,05:32.01,,1,13,NaN,...,,,Mid,,https://nsg.moe.edu.sg/results,Eunoia Junior College,,,02-12,
41,40,296,Velranjan S/O Krishnasamy,MI,NaN,05:42.40,,2,6,NaN,...,,,Mid,,https://nsg.moe.edu.sg/results,Millennia Institute,,,02-12,
42,41,226,Lim Yi Da Alvis,EJC,NaN,05:56.75,,3,3,NaN,...,,,Mid,,https://nsg.moe.edu.sg/results,Eunoia Junior College,,,02-12,


# Production Code for 2022/23 PDFs - Do Not Modify

In [105]:
def extraction(table, master_df):  # latest version to process 2023/22. takes stitchesd list from multiple paged pdfs

    global index, row_index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    session_pattern='S\d\d\-\d\d|S\d\d[A-Z]\-\d\d|S\dl-\d\d|SO\d\-\d\d|SOl-\d\d'
    
    division_pattern='DIVISION'

    
    event = None
    
    names = None
    
    gender = None
    
    temp_df = pd.DataFrame()

    
    # Put extracted info from pdf into a df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
        
    for index, row in df.iterrows(): # find row containing event details and column names
                
        
        if 'Event' in row[0]:
            
            row_string=row[0]
            
            print('row string', row_string)
                                    
            epos = re.search(event_pattern, row_string)
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=row_string[split_start:split_end]
            
            print('event', event)
                                    
            
            if 'BOYS' in row_string or 'Boys' in row_string:
                gender='Male'
                
            if 'GIRLS' in row_string or 'Girls' in row_string:
                gender='Female'    
                
            
            if 'Finals' in row_string:
                round='Finals'
            
            region='Local'    
            
            session_list=re.findall(session_pattern, row_string)
            
            print('session list', session_list)
                
            session=session_list[0]
                
            print('session', session)
            
            event_final = re.sub(session, '', event)     # remove session from string         
            
            division_pos=re.search(division_pattern, row_string)
            
            s_end=division_pos.start()      # adjust the splicing position to only capture the division
            s_start=division_pos.start()-2

            division=row_string[s_start:s_end]
            
            print('division', division)

            print('gender', gender)
     
        
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
            
 #           names[-1]='SESSION'  # change NR column to SESSION column
                        
    
#    if (event and names) is None:  # if it is a stranded page (no header) that is cutoff from the main pdf
                                # Change this code to add iteration over pages. Just append splitted 2nd page.
                                
#            temp_df = master_df.iloc[:0]  # initiate empty df and copy column names
                                    
#            row_index=0   # if no header 
            
#            names = None  # retrieve master df and append instead
            
#            print('here')
            
#            print(table)
                                            
#            temp_df=new_format_parser(row_index, names, table, master_df) # need to concat master_df and temp df
            
#            print('reached this 3rd point for a stranded pdf')
            
#            master_df= pd.concat([master_df, temp_df])  # this concat is not working as temp_df is empty
                    
#            return master_df # exit function

            
    
    # Check pdf format and begin extraction of results
    
    
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
            
        return new_df
            
    elif 'W/G' in columns: # new format     
            
        master_df=new_format_parser(row_index, names, table, master_df) # call parser function
        
        master_df['Session']=session   # add session  NEW
        master_df['Division']=division # add division NEW
        master_df['Event']=event_final # add event NEW
        master_df['Gender']=gender # add event NEW

        
        
        
    ##        master_df= pd.concat([temp_df, master_df])
        
#        print(master_df)
    
    return master_df

In [263]:
# Newest version to ingest 2022 PDFs
# Almost complete except for some formatting effects
# Called by 'extraction' function

def new_format_parser(row_index, names, table, master_df):
    
                
    # Define regex patterns
    
    new_pattern='\d\s.{5,40}?\s(Qfy|Q\s|q\s|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    Q_pattern = '\sQ'
    

    
    stranded = "no"  # flag for stranded pdf pages
    
    NM_pattern = '\bNM'  # No Mark
    
 #   ex_pattern = 'NONE'  # Essentially no exlusions

    
    tag_pattern = '\d\d|\d\d\d'
    
    gap_pattern = ':\s\d'
        
    # Put table that has been processed into a dataframe
    
    df=pd.DataFrame(table)
                
    columns=[]
    
    list=[]
                
    
    # Initialize new df and remove unnecessary columns
    
    if row_index!=0 and names is not None: # check that its coming from not a stranded pdf with no header i.e. new pdf
                    
        df2 = pd.DataFrame(columns=names) 
        
    
        #new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns
        
        new_df = df2    # retain 'REMARKS' and 'NR' columns
    
    else:
        
        print('reached this first point for a stranded pdf')
        
        stranded="yes"
        
        new_df = master_df.iloc[:0] # new_df columns to come from the master df
        
        row_index=-1  # move back row_index for a stranded pdf
        
    
#    for i in range(row_index+1,(len(df)-1)):  # iterate over df - only stranded pdf should start from zero. Previous code - revert to original  


#  check for stranded pdfs with stranded names

    for i in range(row_index+1,(len(df))):  # iterate over df - only stranded pdf should start from index zero    
    
        print('table[i]', table[i])
        
        print('LOOK', re.search('^\d', table[i]), re.search(NM_pattern, table[i]))

                                            
        
        if not re.search('^\d', table[i]) and not re.search(NM_pattern, table[i]): # look for stranded parts of names (does not begin with a digit). But need to test for standed names in stranded pdf
        
            print('checkpoint')
        
            try:  # remove try block to revert back to previous working state
                
                print('checkpoint 2')
                
                print('row[1]', row[1], 'table[i]', table[i])
                                                
                row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name. Stranded pdf will throw and error here. [1] refers to index position 1 corresponding to name
                                                                                
                new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                
                pad_value = ' '
                pad_size = 12 - len(row)

                padded = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
                                                                        
                new_df.loc[len(new_df)] = padded  # add back amended row
                
                print('padded', padded)
                
                list.append(padded)
                                            
                                                                    
                continue  # then skip to next row
            
            except:
                
                print('reached this second point for a stranded pdf')
                
                pass                 # move on if it is actually a stranded pdf
        
        
                              
        if not re.search('^\d', table[i]) and re.search(NM_pattern, table[i]): # look for names that don't contain a numeric ranking with 'NM'
        
  #      if re.search(NM_pattern, table[i]): # look for 'NM'
        
        
            print('NM XXX', table[i])
                            
            string=table[i] 
            
            tag_pos=re.search(tag_pattern, string)
            
            name = string[:tag_pos.start()]
            
            everything_else = string[tag_pos.start():]
            
            list=everything_else.split(' ')
                        
            list.insert(0, name)
            
            list.insert(0, ' ') # insert space for rank
            
            list.insert(2, ' ') # insert space for Q/q         
            
            list.insert(7, ' ')
            
            list.insert(7, ' ')
            
            list.insert(7, ' ')
            
            list.extend([' '])
                                                    
            new_df.loc[len(new_df)] = list
            
        if re.search('^\d', table[i]) and re.search(NM_pattern, table[i]):  # DNS pattern that begins with a digit
            
            string=table[i] 
            
            print('string', string)
            
            matches=re.findall(tag_pattern, string)  # find second set of digits corresponding to  tag ID
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
            
            
            element = string[:rpos.start()]  # contains rank and name together
            
            rank = element[0:3]  # extract rank only
                        
            name =element[3:]  # extract name only
                 
            everything_else = string[rpos.start():]
            
            list=everything_else.split(' ')
            
            print('what', list)
            
            list.insert(0, name)
                                    
            list.insert(0, rank)
            
            list.insert(2, ' ') # insert space for Q/q         
            
            list.insert(7, ' ')
                        
            list.insert(7, ' ')
            
            list.extend([' '])
            
        #    print('DNS pattern', list)
            
            
            new_df.loc[len(new_df)] = list

            
        if table[i].isdigit():  # exit function if it's not a proper line (e.g. number fragments with no content)  
            
            return new_df
        
        
        if i==len(df):     # exit if end of df is reached
            
            return new_df
       
        
        
        # Else split lines per normal format
    
        if not re.search(NM_pattern, table[i]):  # skip those lines matching ex_pattern  
        
            string=table[i] 
                    
            try:
            
                string=string.replace(": ", ":")       #delete gap in timing result
                        
            except:
            
                pass
                    
        
            lpos = re.search('\s', string)
            rpos = re.search(new_pattern, string)
            
            
                                                        
        
        # Splice string to extract name of competitor

            if 'Qfy' not in string:
        
                split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name = string[split_pos_start:split_pos_end]  # extract competitor name
                                
                
           #     print('name no qfy', name)
                
            elif 'Qfy' in string:
                
                split_pos_end=rpos.end()-3      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name
                
            #    print('name got qfy', name)

                
                    
                               
            list=right_string.split(' ')        # put everything coming after the name into a list  
                                                        
                
 #       if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
#            list.extend([' ', ' ', ' ', ' '])
                    
#        elif len(list)==5:
                    
#            list.extend([' ', ' ', ' ', ' '])
                                                            
                
            combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

            row=combined.split(';')   
                                                                                                 

        # check if Q or q between tag and name. If not, fill in with empty space.
        # don't overwrite master df if there is a stranded pdf
                
            if row[2].strip()=='Q' or row[2].strip()=='q' or row[2].strip()=='Qfy' or row[2] is None:
            
                pad_value = ' '
                pad_size = 12 - len(row)

                final_list = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
          #      print('HERE1', final_list)
                                
                new_df.loc[len(new_df)] = final_list


            else:
                
                print('row', row)
                
                row.insert(2, ' ')
                
                
                
                if 'DQ' in string:
                    
                    length=len(row)
                    
                    row.insert(length-2,' ')
                    row.insert(length-2,' ')
                    
          #          print('padded row', row)
            
                pad_value = ' '
                pad_size = 12 - len(row)

                final_list = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
           #     print('HERE2', final_list)
            
                list.append(final_list)
            
                            
                new_df.loc[len(new_df)] = final_list
                                
            
            
            
            
# QAed to this point for a stranded pdf. Need to figure out how to attach stranded row to master df
            
            
#            master_df=new_df      # previous   
    
    return new_df

In [254]:
# Newest version to ingest 2019/2023 PDFs
# Almost complete except for some formatting effects
# Called by 'extraction' function

def new_format_parser(row_index, names, table, master_df):
    
                
    # Define regex patterns
    
    new_pattern='\d\s.{5,40}?\s(Qfy|Q\s|q\s|\d)'  # extract name 
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    Q_pattern = '\sQ'
    

    
    stranded = "no"  # flag for stranded pdf pages
    
    NM_pattern = '\sNM'  # No Mark
    
 #   ex_pattern = 'NONE'  # Essentially no exlusions

    
    tag_pattern = '\d\d\d|\d\d'
    
    gap_pattern = ':\s\d'
        
    # Put table that has been processed into a dataframe
    
    df=pd.DataFrame(table)
                
    columns=[]
    
    list=[]
                
    
    # Initialize new df and remove unnecessary columns
    
    if row_index!=0 and names is not None: # check that its coming from not a stranded pdf with no header i.e. new pdf
                    
        df2 = pd.DataFrame(columns=names) 
        
    
        #new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns
        
        new_df = df2    # retain 'REMARKS' and 'NR' columns
    
    else:
        
        print('reached this first point for a stranded pdf')
        
        stranded="yes"
        
        new_df = master_df.iloc[:0] # new_df columns to come from the master df
        
        row_index=-1  # move back row_index for a stranded pdf
        
    
#    for i in range(row_index+1,(len(df)-1)):  # iterate over df - only stranded pdf should start from zero. Previous code - revert to original  


#  check for stranded pdfs with stranded names

    for i in range(row_index+1,(len(df))):  # iterate over df - only stranded pdf should start from index zero    
    
        print('table[i]', table[i])
        
        print('MATCH', re.search('^\d', table[i]), re.search(NM_pattern, table[i]))

                                            
        
        if not re.search('^\d', table[i]) and not re.search(NM_pattern, table[i]): # look for stranded parts of names (does not begin with a digit). But need to test for standed names in stranded pdf
        
            print('stranded name checkpoint')
        
            try:  # remove try block to revert back to previous working state
                
                print('reattaching stranded name checkpoint')
                
                print('row[1]', row[1], 'table[i]', table[i])
                                                
                row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name. Stranded pdf will throw and error here. [1] refers to index position 1 corresponding to name
                                                                                
                new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                
                pad_value = ' '
                pad_size = 12 - len(row)

                padded = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
                                                                        
                new_df.loc[len(new_df)] = padded  # add back amended row
                
                print('padded full append list', padded)
                
                list.append(padded)
                                            
                                                                    
                continue  # then skip to next row
            
            except:
                
                print('reached this second point for a stranded pdf')
                
                pass                 # move on if it is actually a stranded pdf
        
        ## 'NM' pattern with no numeric ranking ##
                              
        if not re.search('^\d', table[i]) and re.search(NM_pattern, table[i]): 
        
  #      if re.search(NM_pattern, table[i]): # look for 'NM'
        
        
            print('NM XXX', table[i])
                            
            string=table[i] 
            
            tag_pos=re.search(tag_pattern, string)
            
            name = string[:tag_pos.start()]
            
            everything_else = string[tag_pos.start():]
            
            list=everything_else.split(' ')
                        
            list.insert(0, name)
            
            list.insert(0, ' ') # insert space for rank
            
            list.insert(2, ' ') # insert space for Q/q         
            
            list.insert(7, ' ')
            
            list.insert(7, ' ')
            
            list.insert(7, ' ')
            
            list.extend([' '])
            
            print('list', list)
                                                    
            new_df.loc[len(new_df)] = list
            
            
        ## NM pattern that begins with a numeric ranking ##
        
        if re.search('^\d', table[i]) and re.search(NM_pattern, table[i]):  
            
            string=table[i] 
            
            print('NM with ranking string here', string)
            
            matches=re.findall(tag_pattern, string)  # find second set of digits corresponding to  tag ID
            
            print('matches', matches)
                                            
            if len(matches)>=2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                                
                
                print('rpos', rpos)
            
            
            element = string[:rpos.start()]  # contains rank and name together
            
            print('element', element)
            
            rank = element[0:3]  # extract rank only
                        
            name =element[3:]  # extract name only
                 
            everything_else = string[rpos.start():]
            
            list=everything_else.split(' ')
                        
            list.insert(0, name)
                                    
            list.insert(0, rank)
            
            list.insert(2, ' ') # insert space for Q/q         
            
            list.insert(7, ' ')
                        
            list.insert(7, ' ')
            
            list.insert(7, ' ')

            
            list.extend([' '])
            
            print('final list', list)
            
        #    print('DNS pattern', list)
            
            
            new_df.loc[len(new_df)] = list

            
        if table[i].isdigit():  # exit function if it's not a proper line (e.g. number fragments with no content)  
            
            return new_df
        
        
        if i==len(df):     # exit if end of df is reached
            
            return new_df
       
        
        
        ### Normal Splicing of Lines ##
    
        if not re.search(NM_pattern, table[i]):  # skip those lines matching NM_pattern  
        
            string=table[i] 
                        
                    
            try:
            
                string=string.replace(": ", ":")       #delete gap in timing result
                        
            except:
            
                pass
                    
        
            lpos = re.search('\s', string)
            rpos = re.search(new_pattern, string)
            qpos = re.search(Q_pattern, string)

            
            print('lpos', lpos, 'rpos', rpos, 'qpos', qpos)
        
            
            
                                                        
        
        # Splice string to extract name of competitor

            if 'Qfy' not in string:
        
                split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name
                
                # Strip out 'Q' from extracted string
                
          #      try:
                
          #          qpos = re.search(Q_pattern, name)
                    
           #         print('qpos', qpos)
                    
          #          endpos = qpos.start()-1
                    
          #          name = name[:endpos]
                    
                    
                
          #      except:
                    
          #          pass
                
                
           #     print('name no qfy', name)
                
            elif 'Qfy' in string:
                
                split_pos_end=rpos.end()-3      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name
                
            #    print('name got qfy', name)

                
                    
                               
            list=right_string.split(' ')        # put everything coming after the name into a list  
            
            print('LIST', list)
            
            list.insert(6, ' ') # move one over 
                                                        
                
 #       if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
#            list.extend([' ', ' ', ' ', ' '])
                    
#        elif len(list)==5:
                    
#            list.extend([' ', ' ', ' ', ' '])
                                                            
                
            combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

            row=combined.split(';')   
                                                                                                 

        # check if Q or q between tag and name. If not, fill in with empty space.
        # don't overwrite master df if there is a stranded pdf
        
            print('split', row[1].split().pop()) 
            
    #        if row[1].strip()=='Q' or row[1].strip()=='q' or row[1].strip()=='Qfy' or row[1] is None:
            if row[1].split().pop()=='Q':
            
                row[1]=row[1].replace(' Q', ' ')
                
                #row.insert(2, 'Q') # insert 'Q' back into list
           
                row[2] = row[2].replace(' ', 'Q')
            
                print('rowX', row)
        
                pad_value = ' '
                pad_size = 12 - len(row)

                final_list = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
                                                            
                new_df.loc[len(new_df)] = final_list


            else:
                
                print('rowX2', row)
                
                row.insert(2, ' ')
                row.insert(8, ' ')
                
                
                
                if 'DQ' in string:
                    
                    length=len(row)
                    
                    row.insert(length-2,' ')
                    row.insert(length-2,' ')
                    
                if 'DNS' in string:
                    
                    row.insert(7,' ')
                    row.insert(7,' ')
                    row.insert(7,' ')

      #              del row[10]
    
                    row.pop()
    
                if 'DNC' in string:
                    
                    row.insert(7,' ')
                    row.insert(7,' ')
                    row.insert(7,' ')

      #              del row[10]

                    row.pop()

                
                    
            
                pad_value = ' '
                pad_size = 12 - len(row)

                final_list = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
                print('HERE2', final_list)
            
                list.append(final_list)
            
                            
                new_df.loc[len(new_df)] = final_list
                                
            
            
            
            
# QAed to this point for a stranded pdf. Need to figure out how to attach stranded row to master df
            
            
#            master_df=new_df      # previous   
    
    return new_df

In [8]:
def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items


In [264]:
# 2022 ONLY#
# Test iteration over more than one page and multiple files in directory

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022PDF/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1/S01-01_TO_11_deconstructed_22-25.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022/Session_20.pdf"

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022PDF/"

    
    
# Iterate over files in directory

sorted_items=sort_directory(directory)

temp_df=pd.DataFrame()  # initialize empty temp df

master_df=pd.DataFrame()

splitted=None

print(sorted_items)

for file in sorted_items:


    with pdfplumber.open(file) as pdf:
    
        for i in range(len(pdf.pages)):
        
            temp=splitted
        
            page = pdf.pages[i]  # can iterate over different pages
            table=page.extract_table()
            text=page.extract_text()
                
            if i==0:
        
                splitted=text.splitlines()
            
                temp_df=extraction(splitted, temp_df)
            
            else: # stitch list from second page
            
                temp=text.splitlines()
            
                splitted.extend(temp)
            
                temp_df=extraction(splitted, temp_df)
                        
            master_df=pd.concat([master_df, temp_df], axis=0)


            
    #master_df=extraction(splitted, master_df)

                


['Session_1.pdf', 'Session_10.pdf', 'Session_11.pdf', 'Session_12.pdf', 'Session_13.pdf', 'Session_14.pdf', 'Session_14A.pdf', 'Session_15.pdf', 'Session_16.pdf', 'Session_17.pdf', 'Session_18.pdf', 'Session_19.pdf', 'Session_2.pdf', 'Session_20.pdf', 'Session_20A.pdf', 'Session_21.pdf', 'Session_3.pdf', 'Session_4.pdf', 'Session_5.pdf', 'Session_6.pdf', 'Session_7.pdf', 'Session_8.pdf', 'Session_9.pdf']
row string Event: S01-01 Discus - B DIVISION BOYS Finals
event  S01-01 Discus 
session list ['S01-01']
session S01-01
division B 
gender Male
table[i] 1 WONG CHOONG YIN 719 VS Finals 5 38.77 9
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'WONG CHOONG YIN ', ' 719', ' VS', ' Finals', ' 5', ' 38.77', ' 9']
table[i] 2 ONG YI JUN RAYMUS 337 HCI Finals 4 37.56 7
LOOK <re.Match object; span=(0, 1), match='2'> None
row ['2', 'ONG YI JUN RAYMUS ', ' 337', ' HCI', ' Finals', ' 4', ' 37.56', ' 7']
table[i] 3 LEAM TEO JIN TZE 499 RI Finals 14 36.92 6
LOOK <re.Match object; span=(

row string Event: S01-01 Discus - B DIVISION BOYS Finals
event  S01-01 Discus 
session list ['S01-01']
session S01-01
division B 
gender Male
row string Event: S01-02 Long Jump - A DIVISION GIRLS Finals
event  S01-02 Long Jump 
session list ['S01-02']
session S01-02
division A 
gender Female
row string Event: S01-03 1500m - B DIVISION GIRLS Heats
event  S01-03 1500m 
session list ['S01-03']
session S01-03
division B 
gender Female
table[i] 1 FOO SI QING, FAITH Q 333 NHHS h1 7 05:29.48
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'FOO SI QING, FAITH Q', ' ', ' 333', ' NHHS', ' h1', ' 7', ' 05:29.48']
table[i] 2 RUDENKO DARYA Q 516 TMS h1 16 05:31.81
LOOK <re.Match object; span=(0, 1), match='2'> None
row ['2', 'RUDENKO DARYA Q', ' ', ' 516', ' TMS', ' h1', ' 16', ' 05:31.81']
table[i] 3 ISABEL TONG YU WEI Q 170 SNG h2 13 05:28.46
LOOK <re.Match object; span=(0, 1), match='3'> None
row ['3', 'ISABEL TONG YU WEI Q', ' ', ' 170', ' SNG', ' h2', ' 13', ' 05:28.46']
table[i]

row string Event: S01-01 Discus - B DIVISION BOYS Finals
event  S01-01 Discus 
session list ['S01-01']
session S01-01
division B 
gender Male
row string Event: S01-02 Long Jump - A DIVISION GIRLS Finals
event  S01-02 Long Jump 
session list ['S01-02']
session S01-02
division A 
gender Female
row string Event: S01-03 1500m - B DIVISION GIRLS Heats
event  S01-03 1500m 
session list ['S01-03']
session S01-03
division B 
gender Female
row string Event: S01-04 1500m - B DIVISION BOYS Heats
event  S01-04 1500m 
session list ['S01-04']
session S01-04
division B 
gender Male
table[i] 1 NICHOLAS KOH SHENG KAI Q 230 DY h1 4 04:50.52
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'NICHOLAS KOH SHENG KAI Q', ' ', ' 230', ' DY', ' h1', ' 4', ' 04:50.52']
table[i] 2 RAY CHEN KAIYI Q 340 HCI h1 7 04:52.89
LOOK <re.Match object; span=(0, 1), match='2'> None
row ['2', 'RAY CHEN KAIYI Q', ' ', ' 340', ' HCI', ' h1', ' 7', ' 04:52.89']
table[i] 3 DOMINIC LIAU ZE MIN Q 681 VS h2 7 04:40.85


row string Event: S01-01 Discus - B DIVISION BOYS Finals
event  S01-01 Discus 
session list ['S01-01']
session S01-01
division B 
gender Male
row string Event: S01-02 Long Jump - A DIVISION GIRLS Finals
event  S01-02 Long Jump 
session list ['S01-02']
session S01-02
division A 
gender Female
row string Event: S01-03 1500m - B DIVISION GIRLS Heats
event  S01-03 1500m 
session list ['S01-03']
session S01-03
division B 
gender Female
row string Event: S01-04 1500m - B DIVISION BOYS Heats
event  S01-04 1500m 
session list ['S01-04']
session S01-04
division B 
gender Male
table[i] 1 NICHOLAS KOH SHENG KAI Q 230 DY h1 4 04:50.52
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'NICHOLAS KOH SHENG KAI Q', ' ', ' 230', ' DY', ' h1', ' 4', ' 04:50.52']
table[i] 2 RAY CHEN KAIYI Q 340 HCI h1 7 04:52.89
LOOK <re.Match object; span=(0, 1), match='2'> None
row ['2', 'RAY CHEN KAIYI Q', ' ', ' 340', ' HCI', ' h1', ' 7', ' 04:52.89']
table[i] 3 DOMINIC LIAU ZE MIN Q 681 VS h2 7 04:40.85


row string Event: S01-01 Discus - B DIVISION BOYS Finals
event  S01-01 Discus 
session list ['S01-01']
session S01-01
division B 
gender Male
row string Event: S01-02 Long Jump - A DIVISION GIRLS Finals
event  S01-02 Long Jump 
session list ['S01-02']
session S01-02
division A 
gender Female
row string Event: S01-03 1500m - B DIVISION GIRLS Heats
event  S01-03 1500m 
session list ['S01-03']
session S01-03
division B 
gender Female
row string Event: S01-04 1500m - B DIVISION BOYS Heats
event  S01-04 1500m 
session list ['S01-04']
session S01-04
division B 
gender Male
row string Event: S01-05 100m - C DIVISION GIRLS Heats
event  S01-05 100m 
session list ['S01-05']
session S01-05
division C 
gender Female
table[i] 1 CHLOE CHEE EN-YA Q 245 MGS h2 3 00:13.34 +0.9
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'CHLOE CHEE EN-YA Q', ' ', ' 245', ' MGS', ' h2', ' 3', ' 00:13.34', ' +0.9']
table[i] 2 SOH HWEI EN, RACHEL (SU Q 282 NYGH h3 1 00:13.37 +1.1
LOOK <re.Match object; s

...

row string Event: S08-01 Long Jump - C DIVISION GIRLS Heats
event  S08-01 Long Jump 
session list ['S08-01']
session S08-01
division C 
gender Female
table[i] MICHELLE CHOO EN Qfy 110 HIJ Flight1 10
LOOK None None
checkpoint
checkpoint 2
reached this second point for a stranded pdf
row ['MICHELLE', 'CHOO EN Qfy 110 HIJ Flight', ' 1', ' 10']
table[i] LIEW JAY YEE, ESTHER Qfy 294 NJC Flight1 17
LOOK None None
checkpoint
checkpoint 2
row[1] CHOO EN Qfy 110 HIJ Flight table[i] LIEW JAY YEE, ESTHER Qfy 294 NJC Flight1 17
padded ['MICHELLE', 'CHOO EN Qfy 110 HIJ FlightLIEW JAY YEE, ESTHER Qfy 294 NJC Flight1 17', ' ', ' 1', ' 10', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
table[i] YAP LI LING Qfy 300 NJC Flight1 8
LOOK None None
checkpoint
checkpoint 2
row[1] CHOO EN Qfy 110 HIJ FlightLIEW JAY YEE, ESTHER Qfy 294 NJC Flight1 17 table[i] YAP LI LING Qfy 300 NJC Flight1 8
padded ['MICHELLE', 'CHOO EN Qfy 110 HIJ FlightLIEW JAY YEE, ESTHER Qfy 294 NJC Flight1 17YAP LI LING Qfy 300 NJC Flight1 8', ' ',

row string Event: S08-01 Long Jump - C DIVISION GIRLS Heats
event  S08-01 Long Jump 
session list ['S08-01']
session S08-01
division C 
gender Female
row string Event: S08-02 Discus - B DIVISION GIRLS Finals
event  S08-02 Discus 
session list ['S08-02']
session S08-02
division B 
gender Female
table[i] 1 NEO EE NIN 184 SNG Finals 8 28.56 9
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'NEO EE NIN ', ' 184', ' SNG', ' Finals', ' 8', ' 28.56', ' 9']
table[i] 2 HARTHI D/O MANOHARAN 62 CG Finals 16 25.97 7
LOOK <re.Match object; span=(0, 1), match='2'> None
row ['2', 'HARTHI D/O MANOHARAN ', ' 62', ' CG', ' Finals', ' 16', ' 25.97', ' 7']
table[i] 3 EMMA TAN YI MIN 166 SNG Finals 6 25.68 6
LOOK <re.Match object; span=(0, 1), match='3'> None
row ['3', 'EMMA TAN YI MIN ', ' 166', ' SNG', ' Finals', ' 6', ' 25.68', ' 6']
table[i] 4 NADYA WONG TIAN XIN 79 CG Finals 3 23.91 5
LOOK <re.Match object; span=(0, 1), match='4'> None
row ['4', 'NADYA WONG TIAN XIN ', ' 79', ' CG', ' Fi

row string Event: S08-01 Long Jump - C DIVISION GIRLS Heats
event  S08-01 Long Jump 
session list ['S08-01']
session S08-01
division C 
gender Female
row string Event: S08-02 Discus - B DIVISION GIRLS Finals
event  S08-02 Discus 
session list ['S08-02']
session S08-02
division B 
gender Female
row string Event: S08-03 Pole Vault - C DIVISION BOYS Finals
event  S08-03 Pole Vault 
session list ['S08-03']
session S08-03
division C 
gender Male
row string Event: S08-04 5000m - A DIVISION BOYS Heats
event  S08-04 5000m 
session list ['S08-04']
session S08-04
division A 
gender Male
row string Event: S08-05 3000m - A DIVISION GIRLS Heats
event  S08-05 3000m 
session list ['S08-05']
session S08-05
division A 
gender Female
table[i] 1 FAITH ZHEN FORD Q 196 VJC h1 5 12:22.48
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'FAITH ZHEN FORD Q', ' ', ' 196', ' VJC', ' h1', ' 5', ' 12:22.48']
table[i] 2 NG KAI YI Q 149 RI h1 15 12:25.28
LOOK <re.Match object; span=(0, 1), match='2'> N

row string Event: S09-01 Triple Jump - A DIVISION GIRLS Finals
event  S09-01 Triple Jump 
session list ['S09-01']
session S09-01
division A 
gender Female
table[i] 1 ZHONG CHUHAN 103 HCI Finals 6 11.77 9
LOOK <re.Match object; span=(0, 1), match='1'> None
row ['1', 'ZHONG CHUHAN ', ' 103', ' HCI', ' Finals', ' 6', ' 11.77', ' 9']
table[i] 2 CHANDRU BHAVIKA 160 SSP Finals 4 10.98 7
LOOK <re.Match object; span=(0, 1), match='2'> None
row ['2', 'CHANDRU BHAVIKA ', ' 160', ' SSP', ' Finals', ' 4', ' 10.98', ' 7']
table[i] 3 NG JING NI 162 SSP Finals 11 10.79 6
LOOK <re.Match object; span=(0, 1), match='3'> None
row ['3', 'NG JING NI ', ' 162', ' SSP', ' Finals', ' 11', ' 10.79', ' 6']
table[i] 4 YUNN BRITNEY AYE KYAW 133 NJC Finals 12 10.15 5
LOOK <re.Match object; span=(0, 1), match='4'> None
row ['4', 'YUNN BRITNEY AYE KYAW ', ' 133', ' NJC', ' Finals', ' 12', ' 10.15', ' 5']
table[i] 5 POR SZE MUN 128 NJC Finals 15 10.06 4
LOOK <re.Match object; span=(0, 1), match='5'> None
row ['5', 'P

row string Event: S09-01 Triple Jump - A DIVISION GIRLS Finals
event  S09-01 Triple Jump 
session list ['S09-01']
session S09-01
division A 
gender Female
row string Event: S09-02 Javelin - C DIVISION BOYS Finals
event  S09-02 Javelin 
session list ['S09-02']
session S09-02
division C 
gender Male
row string Event: S09-03 Pole Vault - OPENS DIVISION GIRLS Finals
event  S09-03 Pole Vault 
session list ['S09-03']
session S09-03
division S 
gender Female
row string Event: S09-04 110m Hurdles - A DIVISION BOYS Finals
event  S09-04 110m Hurdles 
session list ['S09-04']
session S09-04
division A 
gender Male
row string Event: S09-05 110m Hurdles - B DIVISION BOYS Finals
event  S09-05 110m Hurdles 
session list ['S09-05']
session S09-05
division B 
gender Male
row string Event: S09-06 100m Hurdles - C DIVISION BOYS Finals
event  S09-06 100m Hurdles 
session list ['S09-06']
session S09-06
division C 
gender Male
table[i] 1 CHUA JE-AN, GARRETT 307 RI Finals 4 00:13.94 9 +0.2
LOOK <re.Match obje

In [265]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G,Remarks,NR,Session,Division,Event,Gender
0,1,WONG CHOONG YIN,,719,VS,Finals,5,38.77,9,,,,S01-01,B,Discus,Male
1,2,ONG YI JUN RAYMUS,,337,HCI,Finals,4,37.56,7,,,,S01-01,B,Discus,Male
2,3,LEAM TEO JIN TZE,,499,RI,Finals,14,36.92,6,,,,S01-01,B,Discus,Male
3,4,LEROY ANDRE CHNG CHONG KAI,,695,VS,Finals,12,35.21,5,,,,S01-01,B,Discus,Male
4,5,KEANE WONG JEN QUIN,,585,SJI,Finals,11,34.78,4,,,,S01-01,B,Discus,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,4,QUINN PUAR MIN (PAN MIN),,281,NYGH,Finals,1,00:15.88,5,+0.0,,,S09-09,C,80m Hurdles,Female
4,5,SEM EUNA (SUN YUNYA),,145,SNG,Finals,7,00:16.28,4,+0.0,,,S09-09,C,80m Hurdles,Female
5,6,TAN NGANG TENG XEN (CHEN YANYING),,78,CG,Finals,8,00:16.96,3,+0.0,,,S09-09,C,80m Hurdles,Female
6,7,EDEN SAIGE TAN,,266,NYGH,Finals,2,00:17.16,2,+0.0,,,S09-09,C,80m Hurdles,Female


In [45]:
master_df.to_csv('NSG_2022_final.csv', sep=',', encoding='utf-8-sig', index=False)

In [19]:
print(table)

[['Audited List Published Date: 04-05-2022 09:04:34\nEvent: S20-04 800m - A DIVISION BOYS Finals\nSchools National Record S Pandian BTVI 01:52.37s 1985\nSchools National Record Thiruben S/O Thana Rajan NJC (ET)01:55.40s 2017\nChampionships Record S Pandian BTVI 01:54.5s 1985\nChampionships Record Thiruben S/O Thana Rajan NJC (ET)01:55.40s 2017', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], ['Audited List Published Date: 04-05-2022 09:04:34\nEvent: S20-04 800m - A DIVISION BOYS Finals\nSchools National Record S Pandian BTVI 01:52.37s 1985\nSchools National Record Thiruben S/O Thana Rajan NJC (ET)01:55.40s 2017\nChampionships Record S Pandian BTVI 01:54.5s 1985\nChampionships Record Thiruben S/O Thana Rajan NJC (ET)01:55.40s 2017', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], [None, '', '', 'Audited List Published Date: 04-05-

In [208]:
# 2023 ONLY #
# Test iteration over more than one page and multiple files in directory

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1/S01-01_TO_11_deconstructed_22-25.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022/Session_20.pdf"

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1"

    
    
# Iterate over files in directory

sorted_items=sort_directory(directory)

temp_df=pd.DataFrame()  # initialize empty temp df

master_df=pd.DataFrame()

splitted=None

print(sorted_items)

for file in sorted_items:
    
    print(file)


    with pdfplumber.open(file) as pdf:
    
        for i in range(len(pdf.pages)):
        
            temp=splitted
        
            page = pdf.pages[i]  # can iterate over different pages
            table=page.extract_table()
            text=page.extract_text()
                
            if i==0:
        
                splitted=text.splitlines()
            
                temp_df=extraction(splitted, temp_df)
            
            else: # stitch list from second page
            
                temp=text.splitlines()
            
                splitted.extend(temp)
            
                temp_df=extraction(splitted, temp_df)
                        
            master_df=pd.concat([master_df, temp_df], axis=0)


            
    #master_df=extraction(splitted, master_df)

                


['S01-01_TO_11_deconstructed_1-1.pdf', 'S01-01_TO_11_deconstructed_10-13.pdf', 'S01-01_TO_11_deconstructed_14-17.pdf', 'S01-01_TO_11_deconstructed_18-21.pdf', 'S01-01_TO_11_deconstructed_2-2.pdf', 'S01-01_TO_11_deconstructed_22-25.pdf', 'S01-01_TO_11_deconstructed_26-end.pdf', 'S01-01_TO_11_deconstructed_3-3.pdf', 'S01-01_TO_11_deconstructed_4-4.pdf', 'S01-01_TO_11_deconstructed_5-5.pdf', 'S01-01_TO_11_deconstructed_6-7.pdf', 'S01-01_TO_11_deconstructed_8-9.pdf']
S01-01_TO_11_deconstructed_1-1.pdf
row string Event: S0l-01 High Jump - B DIVISION GIRLS Finals r
event  S0l-01 High Jump 
session list ['S0l-01']
session S0l-01
division B 
gender Female
table[i] 1 BOBBI VICTORIA YANG YEN 117 HIJ Finals 4 1.48 9
MATCH <re.Match object; span=(0, 1), match='1'> None
lpos <re.Match object; span=(1, 2), match=' '> rpos <re.Match object; span=(0, 27), match='1 BOBBI VICTORIA YANG YEN 1'> qpos None
LIST ['117', 'HIJ', 'Finals', '4', '1.48', '9']
split YEN
rowX2 ['1', 'BOBBI VICTORIA YANG YEN ', ' 1

row string Event: S0l-07 100m - B DIVISION BOYS Heats SLFinals on
event  S0l-07 100m 
session list ['S0l-07']
session S0l-07
division B 
gender Male
table[i] 1 BRAYDEN CHAN WEI JIE Q 472 RI h4 1 00:11.25 -0.2
MATCH <re.Match object; span=(0, 1), match='1'> None
lpos <re.Match object; span=(1, 2), match=' '> rpos <re.Match object; span=(0, 25), match='1 BRAYDEN CHAN WEI JIE Q '> qpos <re.Match object; span=(22, 24), match=' Q'>
LIST ['', '472', 'RI', 'h4', '1', '00:11.25', '-0.2']
split Q
rowX ['1', 'BRAYDEN CHAN WEI JIE ', 'Q', ' 472', ' RI', ' h4', ' 1', ' 00:11.25', '  ', ' -0.2']
table[i] 2 SONG EN XU REAGAN Q 86 ACS(!) h2 5 00:11.41 -0.1
MATCH <re.Match object; span=(0, 1), match='2'> None
lpos <re.Match object; span=(1, 2), match=' '> rpos <re.Match object; span=(0, 22), match='2 SONG EN XU REAGAN Q '> qpos <re.Match object; span=(19, 21), match=' Q'>
LIST ['', '86', 'ACS(!)', 'h2', '5', '00:11.41', '-0.1']
split Q
rowX ['2', 'SONG EN XU REAGAN ', 'Q', ' 86', ' ACS(!)', ' h2', ' 5

ValueError: cannot set a row with mismatched columns

In [184]:
# 2023 ONLY #
# NEW TEST CODE#

# Test iteration over more than one page and multiple files in directory

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1/S01-01_TO_11_deconstructed_10-13.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022/Session_20.pdf"

#directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1"

    
    
# Iterate over files in directory

#sorted_items=sort_directory(directory)

temp_df=pd.DataFrame()  # initialize empty temp df

master_df=pd.DataFrame()

splitted=None

#print(sorted_items)

#for file in sorted_items:
    
#    print(file)


with pdfplumber.open(file) as pdf:
    
    for i in range(len(pdf.pages)):
        
        temp=splitted
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()
                
        if i==0:
        
            splitted=text.splitlines()
            
     #       temp_df=extraction(splitted, temp_df)
            
        else: # stitch following pages to create one continuous page
            
            temp=text.splitlines()
            
            splitted.extend(temp)
            
       #     temp_df=extraction(splitted, temp_df)
                        
       # master_df=pd.concat([master_df, temp_df], axis=0)

print('SPLITTED', splitted)

# Correct formatting errors in source data

for j in range(len(splitted)):
    
    if splitted[j] == '24 sA UZE MARTIN J EAN-RAYMOND Q 550 s G hl 2 4 00 :12.23 +0 .4':
        splitted[j] = '24 SAUZE MARTIN JEAN-RAYMOND Q 550 s G hl 2 4 00:12.23 +0.4'
               
    if splitted[j] == '25 TE 0 LIN 581 SJI (I NT) h8 8 00 :12.24 -0. 1':
        splitted[j] = '25 TEO LIN 581 SJI(INT) h8 8 00:12.24 -0.1'
        
    if splitted[j] == '28 TA N JING KHAI 407 NC H hl 1 6 00 :12.27 +0 .1':
        splitted[j] = '28 TAN JING KHAI 407 NC H hl 1 6 00 :12.27 +0.1'
        
    if splitted[j] == '29 LI M SONG CHERN JAYDEN 45 AC S(B R) hl 1 7 00 :12.28 +0 .1':
        splitted[j] = '29 LIM SONG CHERN JAYDEN 45 ACS(BR) hl 1 7 00:12.28 +0.1'
        
    if splitted[j] == '33 H UDSON LIN 26 AI s hl 0 5 00 : 12.45 +0 .2':
        splitted[j] = '33 HUDSON LIN 26 AI s hl 0 5 00:12.45 +0.2'
        
    if splitted[j] == '34 M UHAMMAD RIZQ SYAHEED BIN 645 TM s hl 7 00 : 12.46 +o .1':
        splitted[j] = '34 MUHAMMAD RIZQ SYAHEED BIN 645 TM s hl 7 00:12.46 +0.1'

    if splitted[j] == '35 PAN G CHENG HE NG 312 HY hl 1 5 00 : 12.46 +0 .1':
        splitted[j] = '35 PANG CHENG HENG 312 HY hl 1 5 00:12.46 +0.1'

    if splitted[j] == '36 JA RREL KHOO YU FENG 638 TM s h2 3 00 : 12.47 -0. 1':
        splitted[j] = '36 JARREL KHOO YU FENG 638 TM s h2 3 00:12.47 -0.1'

    if splitted[j] == '38 w EI ZE AN 629 TK h5 7 00 : 12.49 +o .1':
        splitted[j] = '38 WEI ZE AN 629 TK h5 7 00:12.49 +0.1'

    if splitted[j] == '39 KAYDEN LOW 691 vs h8 4 00 :12.50 -0. 1':
        splitted[j] = '39 KAYDEN LOW 691 vs h8 4 00:12.50 -0.1'

    if splitted[j] == '40 C HUA WEE TION G MELVIN 306 HY h9 4 00 :12.51 +0 .2':
        splitted[j] = '40 CHUA WEE TIONG MELVIN 306 HY h9 4 00:12.51 +0.2'

    if splitted[j] == '41 M UHAMMAD ARFA N BIN NOOR 366 JY h6 4 00 :12.53 +0 .3':
        splitted[j] = '41 MUHAMMAD ARFAN BIN NOOR 366 JY h6 4 00:12.53 +0.3'

    if splitted[j] == '42 0 NG HAO EN ZA CHARY 621 TK hl 1 8 00 :12.56 +o .1':
        splitted[j] = '42 ONG HAO EN ZACHARY 621 TK hl 1 8 00:12.56 +0.1'

    if splitted[j] == '43 TA N YI XI CAEDM ON 429 NA S hl 2 2 00 :12.57 +o .4':
        splitted[j] = '43 TAN YI XI CAEDMON 429 NA S hl 2 2 00:12.57 +0.4'

    if splitted[j] == '44 PAN G JING WEN LUCAS 666 us hl 4 5 00 :12.58 +0 .1':
        splitted[j] = '44 PANG JING WEN LUCAS 666 us hl 4 5 00:12.58 +0.1'

    if splitted[j] == '45 EVA N LUAH KAY LE 140 BT V hl 6 00 :12.59 +0 .1':
        splitted[j] = '45 EVAN LUAH KAY LE 140 BT V hl 6 00:12.59 +0.1'

    if splitted[j] == '46 IA N LEE RAY MIN G 557 SJ h6 3 00 :12.59 +o .3':
        splitted[j] = '46 IAN LEE RAY MIN G 557 SJ h6 3 00:12.59 +0.3'

    if splitted[j] == '47 A QEEL FARHAN BI N KHAIRUL 430 NV h7 6 00 :12.60 0. 0':
        splitted[j] = '47 AQEEL FARHAN BIN KHAIRUL 430 NV h7 6 00:12.60 0.0'

    if splitted[j] == '48 VE NKATACHALAPA THY NAREN 649 TM s hl 2 6 00 :12.60 +o .4':
        splitted[j] = '48 VENKATACHALAPA THY NAREN 649 TM s hl 2 6 00:12.60 +0.4'

    if splitted[j] == '49 B RANDON TEO G UAN HAO 296 HSC h8 1 00 :12.64 -0. 1':
        splitted[j] = '49 BRANDON TEO GUAN HAO 296 HSC h8 1 00:12.64 -0.1'
        
    if splitted[j] == '50 wO NG QIWEN KAV ENN 409 NC H h5 5 00 :12.64 +o .1':
        splitted[j] = '50 WONG QIWEN KAV ENN 409 NC H h5 5 00:12.64 +0.1'

    if splitted[j] == '51 LI M JUN JIE 435 NV hl 4 3 00 :12.65 +0 .1':
        splitted[j] = '51 LIM JUN JIE 435 NV hl 4 3 00:12.65 +0.1'

    if splitted[j] == '52 YA K YUN CHEN, JO VAN 246 D HS h7 7 00 :12.66 0. 0':
        splitted[j] = '52 YAK YUN CHEN, JO VAN 246 D HS h7 7 00:12.66 0.0'

    if splitted[j] == '53 TA N WEI XIAN 14 A DSS h8 6 00 :12.69 -0. 1':
        splitted[j] = '53 TAN WEI XIAN 14 A DSS h8 6 00:12.69 -0.1'

    if splitted[j] == '54 TI MOTHYWONG WEN FENG 97 AE s hl 4 4 00 :12.72 +o .1':
        splitted[j] = '54 TIMOTHY WONG WEN FENG 97 AE s hl 4 4 00:12.72 +0.1'

    if splitted[j] == '55 y OW HOI KIT, LUCA S 578 SJ hl 4 2 00 :12.73 +0 .1':
        splitted[j] = '55 YOW HOI KIT, LUCAS 578 SJ hl 4 2 00:12.73 +0.1'
        
    if splitted[j] == '56 LEE ZHE ANN 362 JY h7 5 00 :12.77 0. 0':
        splitted[j] = '56 LEE ZHE ANN 362 JY h7 5 00:12.77 0.0'
  
    if splitted[j] == '57 IL HAN FAHEEM BI N ZAINAL 27 AIs h4 4 00 :12.82 -0. 2':
        splitted[j] = '57 IL HAN FAHEEM BIN ZAINAL 27 AIS h4 4 00:12.82 -0.2'

    if splitted[j] == '58 FO0 SHIN WEI KEA NE 431 NV h6 1 00 :12.82 +0 .3':
        splitted[j] = '58 FOO SHIN WEI KEANE 431 NV h6 1 00:12.82 +0.3'
        
    if splitted[j] == '59 LOH WEN JUN 117 BV h9 8 00 :12.83 +0 .2':
        splitted[j] = '59 LOH WEN JUN 117 BV h9 8 00:12.83 +0.2'
   
    if splitted[j] == '60 NG IA WEN CHEN 235 DY hl 4 00 :12.92 +o .1':
        splitted[j] = '60 NGIA WEN CHEN 235 DY hl 4 00:12.92 +0.1'

    if splitted[j] == '61 sA MUEL ONG HO NGYU 206 cc s hl 0 8 00 :12.93 +o .2':
        splitted[j] = '61 SAMUEL ONG HONG YU 206 CCS h10 8 00:12.93 +0.2'

    if splitted[j] == '62 LOW CHUN KAI 8 A DSS hl 3 7 00 :12.96 -0. 2':
        splitted[j] = '62 LOW CHUN KAI 8 ADSS hl 3 7 00:12.96 -0.2'

    if splitted[j] == '63 JO NAS TUNG JIE JUN 387 MS H h4 6 00 :12.98 -0. 2':
        splitted[j] = '63 JONAS TUNG JIE JUN 387 MS H h4 6 00:12.98 -0.2'

    if splitted[j] == '64 TI TUS ANG 422 NJC hl 5 00 :13.01 +o .1':
        splitted[j] = '64 TITUS ANG 422 NJC hl 5 00:13.01 +0.1'

    if splitted[j] == '65 TH AW ZIN 00 742 YCs h2 7 00 :13.03 -0. 1':
        splitted[j] = '65 THAW ZIN OO 742 YCS h2 7 00:13.03 -0.1'

    if splitted[j] == '66 IS AIAH LING EN JI E 386 MS H hl 0 2 00 :13.03 +0 .2':
        splitted[j] = '66 ISAIAH LING EN JIE 386 MSH h10 2 00:13.03 +0.2'


    if splitted[j] == '66 IS AIAH LING EN JI E 386 MS H hl 0 2 00 :13.03 +0 .2':
        splitted[j] = '66 ISAIAH LING EN JIE 386 MSH h10 2 00:13.03 +0.2'

    if splitted[j] == '67 LI M ZHIRUI, ETHAN 116 BV h4 3 00 :13.04 -0. 2':
        splitted[j] = '67 LIM ZHIRUI, ETHAN 116 BV h4 3 00:13.04 -0.2'
  
    if splitted[j] == '67 LI M ZHIRUI, ETHAN 116 BV h4 3 00 :13.04 -0. 2':
        splitted[j] = '67 LIM ZHIRUI, ETHAN 116 BV h4 3 00:13.04 -0.2'

    if splitted[j] == '68 SE NTHIL KUMAR JERRIL VIVEK 738 YCs hl 1 1 00 :13.05 +0 .1':
        splitted[j] = '68 SENTHIL KUMAR JERRIL VIVEK 738 YCS h11 1 00:13.05 +0.1'


master_df=extraction(splitted, master_df)

                


SPLITTED ['Updated On: 10-04-2023 Audited List', '4:52 PM', 'Event: S0l-07 100m - B DIVISION BOYS Heats SLFinals on', '505 SLFinals', 'on 505', 'Schools National Record Mak Lee Ren ACS(!) (ET)00: 10.59s 2022', 'Championships Record Hong Jinsheng JT 00: 10.8s 1985', 'Championships Record Joshua Chua Hanwei RI (ET)00: 10.87s 2016', 'POS Competitor Q Tag Team Heat Ln Result Pts W/G Remarks NR', '1 BRAYDEN CHAN WEI JIE Q 472 RI h4 1 00:11.25 -0.2', '2 SONG EN XU REAGAN Q 86 ACS(!) h2 5 00:11.41 -0.1', '3 EMIR BIN MUHAMMAD RASHID Q 532 SSP h2 1 00:11.68 -0.1', '4 LOW WEI YI DILLON Q 118 BV h3 3 00: 11.692 -0.6 11.6917', '5 TONG XIAN YAO Q 89 ACS(I) hl 2 00: 11.692 +0.1 11.6920', '6 MIKAIL EMRE WIJAYA Q 596 SPS h4 5 00:11.88 -0.2', '7 TONG ZONG WEI Q 351 HCI hl 3 00: 11.91 +0.1', '8 VEREK CHUA HAO EN Q 391 MSH h9 1 00:11.92 +0.2', '9 JAVEN TEO YI KIAT Q 591 SPS hl0 1 00: 11.94 +0.2', '10 SEAN CHUA JAYANDRAN Q 604 SPS h9 2 00:11.95 +0.2', '11 LIM HAI LE MARCUS Q 613 TK h3 1 00: 11.96 -0.6', '

ValueError: cannot set a row with mismatched columns

In [255]:
# 2019 ONLY #
# NEW TEST CODE#

# Test iteration over more than one page and multiple files in directory

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/sectrack_audit_2019/Session 1_1-3/Session 1_23-23.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022/Session_20.pdf"

#directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1"

    
    
# Iterate over files in directory

#sorted_items=sort_directory(directory)

temp_df=pd.DataFrame()  # initialize empty temp df

master_df=pd.DataFrame()

splitted=None

#print(sorted_items)

#for file in sorted_items:
    
#    print(file)


with pdfplumber.open(file) as pdf:
    
    for i in range(len(pdf.pages)):
        
        temp=splitted
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()
                
        if i==0:
        
            splitted=text.splitlines()
            
     #       temp_df=extraction(splitted, temp_df)
            
        else: # stitch following pages to create one continuous page
            
            temp=text.splitlines()
            
            splitted.extend(temp)
            
       #     temp_df=extraction(splitted, temp_df)
                        
       # master_df=pd.concat([master_df, temp_df], axis=0)

print('SPLITTED', splitted)

# Correct formatting errors in source data

for j in range(len(splitted)):
    
    if splitted[j] == '34 ANG TZE YEN, CLAIRE 144 RI h2 4 f -0.1 DNS':
        splitted[j] = '34 ANG TZE YEN, CLAIRE 144 RI h2 4 -0.1 DNS'

    if splitted[j] == '54 C HUA SHYN YIN N, SHERMAINE 28 BV h8 3 00 : 15.56 +0 .7':
        splitted[j] = '54 CHUA SHYN YINN, SHERMAINE 28 BV h8 3 00:15.56 +0.7'

master_df=extraction(splitted, master_df)

                


SPLITTED ['Audited List', 'Event: S01-07 Triple Jump - B DIVISION GIRLS Finals', 'Schools National Record Lynette Lim Enxuan VJC 11.94m 2015', 'Championships Record Lynette Lim Enxuan CG 11.79m 2013', 'POS Competitor Q Tag Team Heat Ln Result Pts W/G Remarks NR', '1 ANN LEE SHYANN WAI 1025 NYGH Finals 12 11.46 9 0.0', '2 LEOW ZITONG, MABEL 889 CG Finals 9 10.79 7 -0.1', '3 CHUNG NING XIN 664 SNG Finals 5 10.58 6 -0.0', '4 FAITH YEONG RUI MIN 750 HIJ Finals 15 10.50 5 0.0', '5 DU MEI QIAN @DU KAIXUAN 1037 NYGH Finals 1 10.29 4 0.0', '6 VEAN LIM EU SUN 735 SSP Finals 3 10.28 3 0.0', '7 TAN JIN JYE, JURNUS 901 CG Finals 7 10.22 2 0.0', '8 RENA EDWARD 675 SNG Finals 13 10.06 1 -0.1', '9 TAN CHLOE 731 SSP Finals 14 9.75 0.0', '10 EVELYN LAI XIN EN 818 NJC Finals 10 9.34 -0.5', '11 KRISTEN LAM ZHI YU 885 CG Finals 4 9.34 0.6', '12 WONG YUAN PING, DELYN 681 SNG Finals 11 9.21 0.0', '13 CHENG LIAN XIN 1029 NYGH Finals 8 9.14 0.0', '14 SIVAKUMAR BRINDHAA 830 NJC Finals 6 8.50 0.0', 'SAUJENYEA',

In [256]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G,Remarks,NR,Session,Division,Event,Gender
0,1,ANN LEE SHYANN WAI,,1025,NYGH,Finals,12,11.46,,9,,0.0,S01-07,B,Triple Jump,Female
1,2,"LEOW ZITONG, MABEL",,889,CG,Finals,9,10.79,,7,,-0.1,S01-07,B,Triple Jump,Female
2,3,CHUNG NING XIN,,664,SNG,Finals,5,10.58,,6,,-0.0,S01-07,B,Triple Jump,Female
3,4,FAITH YEONG RUI MIN,,750,HIJ,Finals,15,10.50,,5,,0.0,S01-07,B,Triple Jump,Female
4,5,DU MEI QIAN @DU KAIXUAN,,1037,NYGH,Finals,1,10.29,,4,,0.0,S01-07,B,Triple Jump,Female
5,6,VEAN LIM EU SUN,,735,SSP,Finals,3,10.28,,3,,0.0,S01-07,B,Triple Jump,Female
6,7,"TAN JIN JYE, JURNUS",,901,CG,Finals,7,10.22,,2,,0.0,S01-07,B,Triple Jump,Female
7,8,RENA EDWARD,,675,SNG,Finals,13,10.06,,1,,-0.1,S01-07,B,Triple Jump,Female
8,9,TAN CHLOE,,731,SSP,Finals,14,9.75,,0.0,,,S01-07,B,Triple Jump,Female
9,10,EVELYN LAI XIN EN,,818,NJC,Finals,10,9.34,,-0.5,,,S01-07,B,Triple Jump,Female


In [257]:
# 2019/2023 ONLY #
# NEW TEST CODE#

# Test iteration over more than one page and multiple files in directory

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1/S01-01_TO_11_deconstructed_4-4.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022/Session_20.pdf"

#directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1"

    
    
# Iterate over files in directory

#sorted_items=sort_directory(directory)

temp_df=pd.DataFrame()  # initialize empty temp df

master_df=pd.DataFrame()

splitted=None

#print(sorted_items)

#for file in sorted_items:
    
#    print(file)


with pdfplumber.open(file) as pdf:
    
    for i in range(len(pdf.pages)):
        
        temp=splitted
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()
                
        if i==0:
        
            splitted=text.splitlines()
            
     #       temp_df=extraction(splitted, temp_df)
            
        else: # stitch following pages to create one continuous page
            
            temp=text.splitlines()
            
            splitted.extend(temp)
            
       #     temp_df=extraction(splitted, temp_df)
                        
       # master_df=pd.concat([master_df, temp_df], axis=0)

print('SPLITTED', splitted)

# Correct formatting errors in source data

for j in range(len(splitted)):
    
    if splitted[j] == '34 ANG TZE YEN, CLAIRE 144 RI h2 4 f -0.1 DNS':
        splitted[j] = '34 ANG TZE YEN, CLAIRE 144 RI h2 4 -0.1 DNS'

    if splitted[j] == '54 C HUA SHYN YIN N, SHERMAINE 28 BV h8 3 00 : 15.56 +0 .7':
        splitted[j] = '54 CHUA SHYN YINN, SHERMAINE 28 BV h8 3 00:15.56 +0.7'

master_df=extraction(splitted, master_df)

                


SPLITTED ['Updated On: 11-04-2023 Audited List', '1:16 PM', 'Event: S01-03A Triple Jump - B DIVISION BOYS Finals', 'Schools National Record Stefan Tseng Ke Chen SSP 15.52m 2006', 'Championships Record Dylan Wong U Fai SSP 14.27m 2014', 'POS Competitor Q Tag Team Heat Ln Result Pts W/G Remarks NR', '1 TAN SHOU YI REI (CHEN 507 RI Finals 17 13.33 9 0.0', 'SHOUYI)', '2 TRISTAN SHANE CHEN 352 HCI Finals 4 12.82 7 0.9', '3 FU SONG 157 CH Finals 12 12.54 6 -1.5', '4 LAU JIA HERN 489 RI Finals 16 12.48 5 0.0', 'j', '5 YANG EN HAO, DANIEL 424 NJC Finals 15 12.45 4 0.0', '6 TELUKULA SOURENDRA 714 vs Finals 10 12.15 3 -0.3', '7 LIM KYAN 698 vs Finals 20 12.07 2 0.0', 'j', '8 LOW MING DAO, JASE 536 SSP Finals 6 11.95 1 0.0', '9 YEN MING ZHEN 425 NJC Finals 5 11.65 -0.3', '10 GOH XUN LEI, JEROEN 159 CH Finals 2 11.65 -1.5', '11 NG JUN JIE 497 RI Finals 6 11.60 0.0', 'j', '12 RAO TIANYU 343 HCI Finals 14 11.51 0.0', '13 XAVIER ALOYSIUS TANG 541 SSP Finals 8 NM', '14 MUHAMMAD ZAHIN BIN ABDUL 538 SSP

In [258]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G,Remarks,NR,Session,Division,Event,Gender
0,1,TAN SHOU YI REI (CHEN SHOUYI),,507,RI,Finals,17,13.33,,9,,0.0,S01-03,B,A Triple Jump,Male
1,2,TRISTAN SHANE CHEN,,352,HCI,Finals,4,12.82,,7,,0.9,S01-03,B,A Triple Jump,Male
2,3,FU SONG,,157,CH,Finals,12,12.54,,6,,-1.5,S01-03,B,A Triple Jump,Male
3,4,LAU JIA HERN j,,489,RI,Finals,16,12.48,,5,,0.0,S01-03,B,A Triple Jump,Male
4,5,"YANG EN HAO, DANIEL",,424,NJC,Finals,15,12.45,,4,,0.0,S01-03,B,A Triple Jump,Male
5,6,TELUKULA SOURENDRA,,714,vs,Finals,10,12.15,,3,,-0.3,S01-03,B,A Triple Jump,Male
6,7,LIM KYAN j,,698,vs,Finals,20,12.07,,2,,0.0,S01-03,B,A Triple Jump,Male
7,8,"LOW MING DAO, JASE",,536,SSP,Finals,6,11.95,,1,,0.0,S01-03,B,A Triple Jump,Male
8,9,YEN MING ZHEN,,425,NJC,Finals,5,11.65,,-0.3,,,S01-03,B,A Triple Jump,Male
9,10,"GOH XUN LEI, JEROEN",,159,CH,Finals,2,11.65,,-1.5,,,S01-03,B,A Triple Jump,Male


# Processing for BigQuery

In [17]:
# Reading and processing output from 2023/24 NSG CSV for uploading into BigQuery


os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

df = pd.read_csv("NSG_2324.csv")


In [18]:
df

,POS,Tag,Competitor,Team,Result,Heat Number,Lane Nnmber,Remarks,Event,Gender,Division,Stage,Year,Competition,Region,Wind m/s,Data Source,SimpleEvent,Remarks_Corrected,Aggr_Event
0,86.0,537.0,A A Ayu Teja Gayatri,SJC,00:18.0,8.0,5.0,CHIJ St. Joseph's Convent,100m,Female,B,Heats,2024,NSG,Local,NaN,PDF,Sprint,NaN,100m
1,NaN,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,DNF,1.0,5.0,DNF Hai Sing Catholic School,1500m,Female,B,Final,2024,NSG,Local,NaN,PDF,Mid,DNF,1500m
2,NaN,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,NaN,3.0,15.0,Hai Sing Catholic School,3000m,Female,B,Heats,2024,NSG,Local,NaN,PDF,Long,NaN,3000m
3,4.0,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,12:10.5,1.0,3.0,Hai Sing Catholic School,3000m,Female,B,Final,2024,NSG,Local,NaN,PDF,Long,NaN,3000m
4,4.0,286.0,AALIYAH ZULAIKHA BINTE ZULKIFLI,HSC,05:30.6,NaN,2.0,NaN,1500m - B DIVISION GIRLS Heats,Female,B,h2,2023,NSG,Local,NaN,PDF,Mid,NaN,1500m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9908,25.0,136.0,ZULFAHMI BIN SANUSI,BBS,11:27.5,NaN,10.0,NaN,3000m - B DIVISION BOYS Heats,Male,B,h1,2023,NSG,Local,NaN,PDF,Long,NaN,3000m
9909,NaN,NaN,NaN,Reason : lane infringement,NaN,NaN,NaN,NaN,400m,Male,C,SF,2024,NSG,Local,NaN,PDF,Sprint,NaN,400m
9910,NaN,NaN,NaN,Reason : 1st to 2nd runner baton passed out of...,NaN,NaN,NaN,NaN,4x400m Relay,Male,B,Heats,2024,NSG,Local,NaN,PDF,Relay,NaN,4x400m Relay
9911,NaN,NaN,NaN,Reason : 1st runner lane infringement @150m curve,NaN,NaN,NaN,NaN,4x400m Relay,Male,B,Heats,2024,NSG,Local,NaN,PDF,Relay,NaN,4x400m Relay


In [19]:
df.rename(
    columns={0: "RANK", 1: "TAG_ID", 2: "NAME", 3: "TEAM", 4: "RESULT", 5: "HEAT", 6: "LANE", 7: "REMARKS.1", 8: "EVENT.1", 
            9: "GENDER", 10: "DIVISION", 11: "STAGE", 12: "DATE", 13: "COMPETITION", 14: "REGION", 15: "WIND", 16: "SOURCE",
            17: "CATEGORY_EVENT", 18: "REMARKS", 19: "EVENT"},
            inplace=True,
            )


In [20]:
df.columns=["RANK", "TAG_ID", "NAME", "TEAM", "RESULT", "HEAT", "LANE", "REMARKS.1", "EVENT.1", "GENDER", 
             "DIVISION", "STAGE", "DATE", "COMPETITION", "REGION", "WIND", "SOURCE", "CATEGORY_EVENT", 
             "REMARKS", "EVENT"]


In [21]:
df

,RANK,TAG_ID,NAME,TEAM,RESULT,HEAT,LANE,REMARKS.1,EVENT.1,GENDER,DIVISION,STAGE,DATE,COMPETITION,REGION,WIND,SOURCE,CATEGORY_EVENT,REMARKS,EVENT
0,86.0,537.0,A A Ayu Teja Gayatri,SJC,00:18.0,8.0,5.0,CHIJ St. Joseph's Convent,100m,Female,B,Heats,2024,NSG,Local,NaN,PDF,Sprint,NaN,100m
1,NaN,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,DNF,1.0,5.0,DNF Hai Sing Catholic School,1500m,Female,B,Final,2024,NSG,Local,NaN,PDF,Mid,DNF,1500m
2,NaN,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,NaN,3.0,15.0,Hai Sing Catholic School,3000m,Female,B,Heats,2024,NSG,Local,NaN,PDF,Long,NaN,3000m
3,4.0,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,12:10.5,1.0,3.0,Hai Sing Catholic School,3000m,Female,B,Final,2024,NSG,Local,NaN,PDF,Long,NaN,3000m
4,4.0,286.0,AALIYAH ZULAIKHA BINTE ZULKIFLI,HSC,05:30.6,NaN,2.0,NaN,1500m - B DIVISION GIRLS Heats,Female,B,h2,2023,NSG,Local,NaN,PDF,Mid,NaN,1500m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9908,25.0,136.0,ZULFAHMI BIN SANUSI,BBS,11:27.5,NaN,10.0,NaN,3000m - B DIVISION BOYS Heats,Male,B,h1,2023,NSG,Local,NaN,PDF,Long,NaN,3000m
9909,NaN,NaN,NaN,Reason : lane infringement,NaN,NaN,NaN,NaN,400m,Male,C,SF,2024,NSG,Local,NaN,PDF,Sprint,NaN,400m
9910,NaN,NaN,NaN,Reason : 1st to 2nd runner baton passed out of...,NaN,NaN,NaN,NaN,4x400m Relay,Male,B,Heats,2024,NSG,Local,NaN,PDF,Relay,NaN,4x400m Relay
9911,NaN,NaN,NaN,Reason : 1st runner lane infringement @150m curve,NaN,NaN,NaN,NaN,4x400m Relay,Male,B,Heats,2024,NSG,Local,NaN,PDF,Relay,NaN,4x400m Relay


In [22]:
df['QUALIFICATION']=''
df['POINTS']=''
df['UNIQUE_ID']=''
df['COUNTRY']=''
df['DICT_RESULTS']=''
df['GROUP']=''
df['TIMESTAMP']=''
df['FREE_FIELD']=''
df['FREE_FIELD2']=''
df['FREE_FIELD3']=''



In [23]:
df = df.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                            'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                            'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


In [24]:
df

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,86.0,537.0,A A Ayu Teja Gayatri,SJC,NaN,00:18.0,,8.0,5.0,NaN,...,NaN,,Sprint,NaN,PDF,NaN,,,,
1,NaN,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,NaN,DNF,,1.0,5.0,NaN,...,NaN,,Mid,NaN,PDF,DNF,,,,
2,NaN,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,NaN,NaN,,3.0,15.0,NaN,...,NaN,,Long,NaN,PDF,NaN,,,,
3,4.0,357.0,Aaliyah Zulaikha Binte Zulkifli,HSC,NaN,12:10.5,,1.0,3.0,NaN,...,NaN,,Long,NaN,PDF,NaN,,,,
4,4.0,286.0,AALIYAH ZULAIKHA BINTE ZULKIFLI,HSC,NaN,05:30.6,,NaN,2.0,NaN,...,NaN,,Mid,NaN,PDF,NaN,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9908,25.0,136.0,ZULFAHMI BIN SANUSI,BBS,NaN,11:27.5,,NaN,10.0,NaN,...,NaN,,Long,NaN,PDF,NaN,,,,
9909,NaN,NaN,NaN,Reason : lane infringement,NaN,NaN,,NaN,NaN,NaN,...,NaN,,Sprint,NaN,PDF,NaN,,,,
9910,NaN,NaN,NaN,Reason : 1st to 2nd runner baton passed out of...,NaN,NaN,,NaN,NaN,NaN,...,NaN,,Relay,NaN,PDF,NaN,,,,
9911,NaN,NaN,NaN,Reason : 1st runner lane infringement @150m curve,NaN,NaN,,NaN,NaN,NaN,...,NaN,,Relay,NaN,PDF,NaN,,,,


In [26]:
df.to_csv('NSG_202324_processed.csv', sep=',', encoding='utf-8-sig', index=False)

## Convert new schema file to old schema in BQ

In [4]:
# Reading and processing output from 2022 NSG CSV for uploading into BigQuery
# Basing on latest SCHEMA taken from Alph's extraction
# Pending my own export code using latest schema
# SUB_EVENT=shot put or javelin under decathlon
# EVENT_CLASS=weight or height for javelin (600g) , hurdles (0.762m or 0.840m) for example
# DATE=DDMM format

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2022/')

df = pd.read_csv("NSG2022_NEW_SCHEMA.csv")


In [5]:
df

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,VENUE,DATE,SUB_EVENT,SESSION,EVENT_CLASS,Remarks_Corrected
0,1,719,WONG CHOONG YIN,VS,NaN,38.77,NaN,Finals,5,NaN,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S0101,NaN,NaN
1,2,337,ONG YI JUN RAYMUS,HCI,NaN,37.56,NaN,Finals,4,NaN,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S0101,NaN,NaN
2,3,499,LEAM TEO JIN TZE,RI,NaN,36.92,NaN,Finals,14,NaN,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S0101,NaN,NaN
3,4,695,LEROY ANDRE CHNG CHONG KAI,VS,NaN,35.21,NaN,Finals,12,NaN,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S0101,NaN,NaN
4,5,585,KEANE WONG JEN QUIN,SJI,NaN,34.78,NaN,Finals,11,NaN,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S0101,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4518,8,15,HAYDEN TAN LI ERN,ACJC,NaN,DNS,NaN,Finals,3,-0.1,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S0904,NaN,NaN
4519,24,431,NG LIN YEE CAYLYNE,PLMGS,NaN,DNS,NaN,SF3,7,1.2,...,NaN,https://nsg.moe.edu.sg/sssc/archives,DNS,NaN,NaN,NaN,NaN,S1005,NaN,NaN
4520,16,250,NUR HAERYL IHSAN BIN CHAIRILYANY,JSS,NaN,12:16.0,NaN,Finals,16,NaN,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S1202,NaN,NaN
4521,23,41,BUKIT VIEW SECONDARY SCHOOL,BIV,NaN,DQ,NaN,SF3,7,NaN,...,NaN,https://nsg.moe.edu.sg/sssc/archives,NaN,NaN,NaN,NaN,NaN,S1508,NaN,NaN


In [6]:
df['GENDER'] = df['GENDER'].replace(regex=r'Men', value='Male')
df['GENDER'] = df['GENDER'].replace(regex=r'Women', value='Female')
df['COMPETITION'] = df['COMPETITION'].replace(regex=r'National School Games', value='NSG')
df['SOURCE'] = df['SOURCE'].replace(regex=r'https://nsg.moe.edu.sg/sssc/archives', value='PDF')



In [7]:
# Drop new columns under latest schema to conform with existing BQ schema

df = df.drop(['Remarks_Corrected', 'EVENT_CLASS', 'SESSION', 'SUB_EVENT', 'DATE', 'VENUE'], axis=1)

In [72]:
#df.rename(columns={'YEAR': 'DATE'}, inplace=True)

In [8]:
# Add back columns required by current BQ schema
# Need to update BQ schema to latest

df['REGION']='Local'
df['FREE_FIELD']=''
df['FREE_FIELD2']=''
df['FREE_FIELD3']=''



In [9]:
# Remove control characters re.sub(r'[\x00-\x1F\x7F]', ' ', value)

def control_char(string):
    
    string=re.sub(r'[\x00-\x1F\x7F]', ' ', string)
    
    return string
    

df['NAME'] = df['NAME'].apply(control_char)


In [10]:
df

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,1,719,WONG CHOONG YIN,VS,NaN,38.77,NaN,Finals,5,NaN,...,NaN,NaN,Throw,NaN,PDF,NaN,NaN,,,
1,2,337,ONG YI JUN RAYMUS,HCI,NaN,37.56,NaN,Finals,4,NaN,...,NaN,NaN,Throw,NaN,PDF,NaN,NaN,,,
2,3,499,LEAM TEO JIN TZE,RI,NaN,36.92,NaN,Finals,14,NaN,...,NaN,NaN,Throw,NaN,PDF,NaN,NaN,,,
3,4,695,LEROY ANDRE CHNG CHONG KAI,VS,NaN,35.21,NaN,Finals,12,NaN,...,NaN,NaN,Throw,NaN,PDF,NaN,NaN,,,
4,5,585,KEANE WONG JEN QUIN,SJI,NaN,34.78,NaN,Finals,11,NaN,...,NaN,NaN,Throw,NaN,PDF,NaN,NaN,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4518,8,15,HAYDEN TAN LI ERN,ACJC,NaN,DNS,NaN,Finals,3,-0.1,...,NaN,NaN,Hurdles,NaN,PDF,NaN,NaN,,,
4519,24,431,NG LIN YEE CAYLYNE,PLMGS,NaN,DNS,NaN,SF3,7,1.2,...,NaN,NaN,Sprint,NaN,PDF,DNS,NaN,,,
4520,16,250,NUR HAERYL IHSAN BIN CHAIRILYANY,JSS,NaN,12:16.0,NaN,Finals,16,NaN,...,NaN,NaN,Long,NaN,PDF,NaN,NaN,,,
4521,23,41,BUKIT VIEW SECONDARY SCHOOL,BIV,NaN,DQ,NaN,SF3,7,NaN,...,NaN,NaN,Relay,NaN,PDF,NaN,NaN,,,


In [51]:
#df = df.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
#                            'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
#                            'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP'])


In [11]:
# Standardise descriptions to be consistent with database

df['EVENT'] = df['EVENT'].replace(regex=r'Hurdles', value='hurdles')
df['EVENT'] = df['EVENT'].replace(regex=r'Walk', value='walk')
df['EVENT'] = df['EVENT'].replace(regex=r'SC', value='steeplechase')
df['EVENT'] = df['EVENT'].replace(regex=r'Jump', value='jump')
df['EVENT'] = df['EVENT'].replace(regex=r'Relay', value='relay')
df['EVENT'] = df['EVENT'].replace(regex=r'Vault', value='vault')
df['EVENT'] = df['EVENT'].replace(regex=r'Put', value='put')
df['DIVISION'] = df['DIVISION'].replace(regex=r'_DIVISION', value='')

mask = df['EVENT'].str.contains(r'walk', na=True)
df.loc[mask, 'CATEGORY_EVENT'] = 'Walk'




In [13]:
df.rename(columns={'YEAR': 'DATE'}, inplace=True)

In [14]:
df.to_csv('NSG_2022_processed.csv', sep=',', encoding='utf-8', index=False)